# Supplemental Code
- Inherent instability of simple DNA repeats shapes an evolutionarily stable distribution of repeat lengths 
- McGinty et al. 2025
- Part 2 of 4

### Load Python libraries and define functions
- Links included for instructions on how to install libraries using pip or conda (if libraries not included with conda)

In [ ]:
import pandas as pd
import numpy as np
import os
import random
import pickle
import gzip
import regex as re
import fastapy
from liftover import ChainFile    # https://pypi.org/project/liftover/
from liftover import get_lifter
import gc
import scipy
from multiprocessing import Process, Manager
manager = Manager()

import kaleido    # https://github.com/plotly/Kaleido
import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
pio.templates.default = "none"

In [ ]:
bases = ['A', 'T', 'G', 'C']
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]]) 
def repeat_frames_RC(input_seq):
    return list(pd.Series([''.join(input_seq*2)[i:len(input_seq)+i] for i in range(len(input_seq))] + [reverse_complement(seq) for seq in [''.join(input_seq*2)[i:len(input_seq)+i] for i in range(len(input_seq))]]).sort_values().drop_duplicates())

flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
def get_sequence(chrom, start, end, genome):
    chrom_list_current = genome.keys()
    start_min = min(start, end); end_max = max(start, end)
    if chrom in chrom_list_current:
        return genome[chrom][int(start_min):int(end_max)]
    elif chrom[3:] in chrom_list_current:
        return genome[chrom[3:]][int(start_min):int(end_max)]
    elif int(chrom[3:]) in chrom_list_current:
        return genome[int(chrom[3:])][int(start_min):int(end_max)]
    else:
        return np.nan

### Load reference genome

In [ ]:
chrom_list = list(range(1,23))
chrom_list_XY = list(range(1,23)) + ['X', 'Y']

#### T2T-CHM13 genome

In [ ]:
CHM13_genome_fasta = list(fastapy.parse('genomes/hs1.fa.gz'))
CHM13_genome = dict()
for seq in CHM13_genome_fasta:
    try:
        name = int(seq.id[3:])
    except:
        name = seq.id[3:]
    if name in chrom_list_XY:
        CHM13_genome[name] = seq.seq.upper()
del CHM13_genome_fasta
gc.collect()
CHM13_genome.keys()

#### hg38 genome
- download "hg38.fa.gz" from http://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/

In [ ]:
hg38_genome_fasta = list(fastapy.parse('genomes/hg38.fa.gz'))
hg38_genome = dict()
for seq in hg38_genome_fasta:
    try:
        name = int(seq.id[3:])
    except:
        name = seq.id[3:]
    if name in chrom_list_XY:
        hg38_genome[name] = seq.seq.upper()
del hg38_genome_fasta
gc.collect()
hg38_genome.keys()

# de novo instability rate measurement

## Prepare de novo mutation database
- Gather de novo data from all available public sources (trio/family sequencing)
- See manuscript for DOI links to these studies
- Place files in directory './denovo/download/'

### de novo data aligned to hg19

#### data from Goldman 2016 (hg19)
- Parent-of-origin-specific signatures of de novo mutations
- phased
- 816 trios

In [ ]:
trio_gold = pd.read_excel('./denovo/download/41588_2016_BFng3597_MOESM69_ESM_hg19.xlsx', usecols = ['Chromosome', 'Start.position', 'Reference', 'Variant', 'parentOfOrigin'])
trio_gold.columns = ['chrom', 'pos', 'ref', 'alt', 'parent']

#### data from Goes et al 2021 (hg19)
- De novo variation in bipolar disorder
- unphased
- 97 trios

In [ ]:
goes2019 = pd.read_excel('./denovo/download/41380_2019_611_MOESM3_ESM.xlsx', usecols = ['chr_bp_ref_alt', 'SNV'])
goes2019[['chrom', 'pos', 'ref', 'alt']] = goes2019['chr_bp_ref_alt'].str.split('_', expand = True)
goes2019 = goes2019.dropna()[['chrom', 'pos', 'ref', 'alt']]
goes2019['chrom'] = ['chr' + str(chrom) for chrom in goes2019['chrom']]

#### data from Yuen et al. 2016 (hg19)
- Genome-wide characteristics of de novo mutations in autism
- 192 trios
- phased

In [ ]:
yuen_trios = pd.read_excel('./denovo/download/41525_2016_BFnpjgenmed201627_MOESM431_ESM.xlsx', sheet_name='Table S4', skiprows = 1, usecols = ['Chromosome', 'Start', 'Reference', 'Allel', 'Parental Origin'])
yuen_trios.columns = ['chrom', 'pos', 'ref', 'alt', 'parent']

#### data from Sasani 2019 (hg19)
- Large, three-generation human families reveal post-zygotic mosaicism and variability in germline mutation accumulation
- phased
- 350 3rd generation offspring, 70 2nd generation offspring (420 genomes)

In [ ]:
dnm_ceph_gen2 = pd.read_csv('./denovo/download/ceph-dnm-manuscript-master/data/second_gen.dnms.txt', sep = '\t', usecols = ['chrom', 'start', 'ref', 'alt', 'paternal_age_at_birth', 'maternal_age_at_birth', 'phase'])
dnm_ceph_gen3 = pd.read_csv('./denovo/download/ceph-dnm-manuscript-master/data/third_gen.dnms.txt', sep = '\t', usecols = ['chrom', 'start', 'ref', 'alt', 'paternal_age_at_birth', 'maternal_age_at_birth', 'phase'])
dnm_ceph_gen2_gon = pd.read_csv('./denovo/download/ceph-dnm-manuscript-master/data/gonosomal.dnms.txt', sep = '\t', usecols = ['chrom', 'start', 'ref', 'alt', 'paternal_age_at_birth', 'maternal_age_at_birth', 'phase'])
dnm_ceph_gen3_gon = pd.read_csv('./denovo/download/ceph-dnm-manuscript-master/data/post-pgcs.dnms.txt', sep = '\t', usecols = ['chrom', 'start', 'ref', 'alt', 'paternal_age_at_birth', 'maternal_age_at_birth', 'phase'])

dnm_ceph = pd.concat([dnm_ceph_gen2, dnm_ceph_gen2_gon, dnm_ceph_gen3, dnm_ceph_gen3_gon])
dnm_ceph['chrom'] = ['chr' + str(chrom) for chrom in dnm_ceph['chrom']]

dnm_ceph.columns = ['chrom', 'pos', 'ref', 'alt', 'paternal_age', 'maternal_age', 'phase']

#### Liftover hg19 data to hg38

In [ ]:
trio_hg19 = pd.concat([trio_gold, goes2019, dnm_ceph, yuen_trios]).reset_index(drop = True)
trio_hg19['pos'] = trio_hg19['pos'].astype(int)

# Liftover to hg38
lo = get_lifter('hg19', 'hg38')
trio_hg19['hg38_lo'] = [lo.convert_coordinate(chrom, pos) for chrom,pos in zip(trio_hg19['chrom'], trio_hg19['pos'])]
trio_hg19['hg38_chr'] = [pos[0][0] if len(pos) >0 else np.nan for pos in trio_hg19['hg38_lo']]
trio_hg19['hg38_pos'] = [pos[0][1] if len(pos) >0 else np.nan for pos in trio_hg19['hg38_lo']]
trio_hg19 = trio_hg19.dropna(subset = ['hg38_chr'], axis=0)
trio_hg19['hg38_chr'] = [int(chrom[3:]) if (chrom[3:] not in ['X', 'Y']) & (len(chrom) < 6) else chrom[3:] if chrom[3:] in ['X', 'Y'] else np.nan for chrom in trio_hg19['hg38_chr']]
trio_hg19 = trio_hg19.dropna(subset = ['hg38_chr'])
trio_hg19['hg38_pos'] = trio_hg19['hg38_pos'].astype(int)
trio_hg19['hg38_base'] = [hg38_genome[chrom][pos-1] for chrom, pos in zip(trio_hg19['hg38_chr'], trio_hg19['hg38_pos'])]
trio_hg19 = trio_hg19.loc[trio_hg19['ref'] == trio_hg19['hg38_base']].copy()
trio_hg19 = trio_hg19[['hg38_chr', 'hg38_pos', 'ref', 'alt', 'parent']].copy()
trio_hg19.columns = ['chrom', 'pos', 'ref', 'alt', 'parent']

### de novo data aligned to hg38

#### data from Halldorsson 2019 in Science (hg38)
- Characterizing mutagenic effects of recombination through a sequence-level genetic map
- phased
- 2976 trios

In [ ]:
trio_set1 = pd.read_csv('./denovo/download/aau1043_DataS5_revision1.tsv', skiprows = 11, sep = '\t', low_memory=False)
trio_set1_ages = pd.read_csv('./denovo/download/aau1043_DataS7.tsv', skiprows = 4, sep = '\t')

trio_set1_ages.index = trio_set1_ages['Proband_id']
trio_set1['Father_age'] = [trio_set1_ages['Father_age'][pro] for pro in trio_set1['Proband_id']]
trio_set1['Mother_age'] = [trio_set1_ages['Mother_age'][pro] for pro in trio_set1['Proband_id']]

trio_set1['Chr'] = [chrom[3:] if chrom.startswith('chr') == True else chrom for chrom in trio_set1['Chr']]
trio_set1['Chr'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in trio_set1['Chr']]

trio_set1 = trio_set1[['Chr', 'Pos', 'Ref', 'Alt', 'Phase_combined', 'Father_age', 'Mother_age']]
trio_set1.columns = ['chrom', 'pos', 'ref', 'alt', 'parent', 'paternal_age', 'maternal_age']

#### data from Jonnson 2017 in Nature (hg38)
- Parental influence on human germline de novo mutations in 1,548 trios from Iceland
- 1548 trios
- phased

In [ ]:
trio_set2 = pd.read_csv('./denovo/download/decode_DNMs.tsv', sep = '\t', usecols = ['Chr', 'Pos_hg38', 'Ref', 'Alt', 'Discordant_in_3_gen_or_mz_twins', 'Fathers_age_at_conception', 'Mothers_age_at_conception', 'Phase_combined'])

trio_set2['Chr'] = [chrom[3:] if chrom.startswith('chr') == True else chrom for chrom in trio_set2['Chr']]
trio_set2['Chr'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in trio_set2['Chr']]

trio_set2.columns = ['chrom', 'pos', 'ref', 'alt', 'discordant', 'paternal_age', 'maternal_age', 'parent']
trio_set2 = trio_set2.loc[trio_set2['discordant'] != 'Discordant']
del trio_set2['discordant']

#### data from AN 2018 in Science (hg38)
- Genome-wide de novo risk score implicates promoter variation in autism spectrum disorder
- unphased
- 1902 quartets (3804 genomes)

In [ ]:
trio_an = pd.read_excel('./denovo/download/aat6576_Table-S2_hg38_notphased.xlsx', skiprows = 1, usecols = ['Chr', 'Pos', 'Ref', 'Alt'])

trio_an['Chr'] = [chrom[3:] if chrom.startswith('chr') == True else chrom for chrom in trio_an['Chr']]
trio_an['Chr'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in trio_an['Chr']]

trio_an.columns = ['chrom', 'pos', 'ref', 'alt']

#### data from Jonsson et al 2021 (hg38)
- Differences between germline genomes of monozygotic twins
- 451 offspring in quads
- 608 offspring in three-generation approach

In [ ]:
# de novo mutations from quads
jonsson_quads = pd.read_csv('./denovo/download/41588_2020_755_MOESM5_ESM.tsv', sep = '\t', usecols = ['Chr', 'Pos', 'Ref', 'Alt', 'Child_Fathers_Age_at_birth', 'Child_Mothers_Age_at_birth'])
jonsson_quads.columns = ['chrom', 'pos', 'ref', 'alt', 'paternal_age', 'maternal_age']

# de novo mutations from three-generation approach
jonsson_3gen = pd.read_csv('./denovo/download/41588_2020_755_MOESM4_ESM.tsv', sep = '\t', usecols = ['Chr', 'Pos', 'Ref', 'Alt'])
jonsson_3gen.columns = ['chrom', 'pos', 'ref', 'alt']

jonsson_quads['chrom'] = [chrom[3:] if chrom.startswith('chr') == True else chrom for chrom in jonsson_quads['chrom']]
jonsson_quads['chrom'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in jonsson_quads['chrom']]

jonsson_3gen['chrom'] = [chrom[3:] if chrom.startswith('chr') == True else chrom for chrom in jonsson_3gen['chrom']]
jonsson_3gen['chrom'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in jonsson_3gen['chrom']]

### Combine all into single de novo database

#### SNVs

In [ ]:
denovo_combined = pd.concat([trio_set1, trio_set2, trio_an, jonsson_3gen, jonsson_quads, trio_hg19]).reset_index(drop = True)
denovo_combined = denovo_combined.loc[(denovo_combined['ref'].str.len() == 1) & (denovo_combined['alt'].str.len() == 1)].copy()
denovo_combined['parent'] = denovo_combined['parent'].str.lower()
denovo_combined['parent'] = denovo_combined['parent'].fillna('unassigned')
denovo_combined['tri'] = [hg38_genome[chrom][pos-2:pos+1] for chrom, pos in zip(denovo_combined['chrom'], denovo_combined['pos'])]

In [ ]:
# Save database
denovo_combined.to_csv('./denovo/all_denovo_snvs_remake.csv', index = False)

In [ ]:
denovo_n_genomes_snv = (816 + 97 + 192 + 420 + 2976+ 1548 + 3804 + 451 + 608) * 2

#### Indels
- note: hg19 files do not contain any indels
- all hg38 files contain indels

In [ ]:
denovo_combined_indel = pd.concat([trio_set1, trio_set2, trio_an, jonsson_3gen, jonsson_quads]).reset_index(drop = True)
denovo_combined_indel = denovo_combined_indel.loc[(denovo_combined_indel['ref'].str.len() != 1) | (denovo_combined_indel['alt'].str.len() != 1)].copy()
denovo_combined_indel['parent'] = denovo_combined_indel['parent'].str.lower()
denovo_combined_indel['parent'] = denovo_combined_indel['parent'].fillna('unassigned')

# Save database
denovo_combined_indel.to_csv('./denovo/all_denovo_indels.csv', index = False)

In [ ]:
denovo_n_genomes_indel = (2976+ 1548 + 3804 + 451 + 608) * 2

In [ ]:
denovo_n_genomes_indel / 2

In [ ]:
denovo_combined_indel = pd.read_csv('./denovo/all_denovo_indels.csv')
denovo_combined_indel

## Calculate de novo substitution rate considering context for A-mononucleotide repeats

In [ ]:
denovo_combined_snv = pd.read_csv('denovo/all_denovo_snvs_remake.csv', low_memory=False)
denovo_n_genomes_snv = (816 + 97 + 192 + 420 + 2976+ 1548 + 3804 + 451 + 608) * 2

In [ ]:
denovo_combined_snv['chrom'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in denovo_combined_snv['chrom']]

In [ ]:
denovo_combined_snv['tri_AB'] = denovo_combined_snv['tri'].str.replace('C', 'B').str.replace('T', 'B').str.replace('G', 'B')
denovo_combined_snv['pos_100kb'] = ((denovo_combined_snv['pos'] / 1e5).astype(int) * 1e5).astype(int)
count_by_100kb = denovo_combined_snv.groupby(['chrom', 'pos_100kb'])['ref'].count()

In [ ]:
triplet_counts_mutregions = dict()
for chrom, pos in count_by_100kb.index:
    seq = hg38_genome[chrom][int(pos):int(pos+1e5)]
    triplet_counts_mutregions[(chrom, pos, 'A')] = pd.Series(re.findall('...', seq), dtype=object).value_counts()
    triplet_counts_mutregions[(chrom, pos, 'B')] = pd.Series(re.findall('...', seq[1:]), dtype=object).value_counts()
    triplet_counts_mutregions[(chrom, pos, 'C')] = pd.Series(re.findall('...', seq[2:]), dtype=object).value_counts()
    print ('\r' + 'finished chr'+str(chrom) + ' '+str(pos), end = ' ')
triplet_counts_mutregions_sum = pd.concat(triplet_counts_mutregions, axis=1).sum(axis=1).reindex(denovo_combined_snv['tri'].value_counts().index).sort_index()

In [ ]:
triplet_counts_mutregions_sum.to_pickle('denovo/triplet_counts_mutregions_sum.pickle')

In [ ]:
triplet_counts_mutregions_sum = pd.read_pickle('denovo/triplet_counts_mutregions_sum.pickle')

In [ ]:
triplet_counts_mutregions_AB = triplet_counts_mutregions_sum.reset_index()
triplet_counts_mutregions_AB['tri_AB'] = triplet_counts_mutregions_AB['tri'].str.replace('C', 'B').str.replace('T', 'B').str.replace('G', 'B')
triplet_counts_mutregions_AB = triplet_counts_mutregions_AB.groupby(['tri_AB'])[0].sum()

In [ ]:
Afusion = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & (denovo_combined_snv['tri_AB'] == 'ABA')]) / triplet_counts_mutregions_AB['ABA'] / denovo_n_genomes_snv
Aexpansion = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & (denovo_combined_snv['tri_AB'] == 'BBB')]) / triplet_counts_mutregions_AB['BBB'] / denovo_n_genomes_snv
A01 = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & ((denovo_combined_snv['tri_AB'] == 'ABB') | (denovo_combined_snv['tri_AB'] == 'BBA'))]) / triplet_counts_mutregions_AB[['ABB', 'BBA']].sum() / denovo_n_genomes_snv
Afission = len(denovo_combined_snv.loc[(denovo_combined_snv['tri_AB'] == 'AAA')]) / triplet_counts_mutregions_AB['AAA'] / denovo_n_genomes_snv
Acontraction = len(denovo_combined_snv.loc[(denovo_combined_snv['tri_AB'] == 'BAB')]) / triplet_counts_mutregions_AB['BAB'] / denovo_n_genomes_snv
A10 = len(denovo_combined_snv.loc[((denovo_combined_snv['tri_AB'] == 'AAB') | (denovo_combined_snv['tri_AB'] == 'BAA'))]) / triplet_counts_mutregions_AB[['AAB', 'BAA']].sum() / denovo_n_genomes_snv

denovo_mut_freq_AB = pd.Series([Afission, Acontraction, A10, Afusion, Aexpansion, A01], index = ['Afission', 'Acontraction', 'A10', 'Afusion', 'Aexpansion', 'A01'])
denovo_mut_freq_AB.to_pickle('denovo/denovo_mut_freq_AB_remake.pickle')

In [ ]:
denovo_mut_freq_AB = pd.read_pickle('denovo/denovo_mut_freq_AB_remake.pickle')
denovo_mut_freq_AB

In [ ]:
triplet_counts_mutregions_CD = triplet_counts_mutregions_sum.reset_index()
triplet_counts_mutregions_CD['tri_CD'] = triplet_counts_mutregions_CD['tri'].str.replace('A', 'D').str.replace('T', 'D').str.replace('G', 'D')
triplet_counts_mutregions_CD = triplet_counts_mutregions_CD.groupby(['tri_CD'])[0].sum()

denovo_combined_snv['tri_CD'] = denovo_combined_snv['tri'].str.replace('A', 'D').str.replace('T', 'D').str.replace('G', 'D')

Cfusion = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & (denovo_combined_snv['tri_CD'] == 'CDC')]) / triplet_counts_mutregions_CD['CDC'] / denovo_n_genomes_snv
Cexpansion = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & (denovo_combined_snv['tri_CD'] == 'DDD')]) / triplet_counts_mutregions_CD['DDD'] / denovo_n_genomes_snv
C01 = len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & ((denovo_combined_snv['tri_CD'] == 'CDD') | (denovo_combined_snv['tri_CD'] == 'DDC'))]) / triplet_counts_mutregions_CD[['CDD', 'DDC']].sum() / denovo_n_genomes_snv
Cfission = len(denovo_combined_snv.loc[(denovo_combined_snv['tri_CD'] == 'CCC')]) / triplet_counts_mutregions_CD['CCC'] / denovo_n_genomes_snv
Ccontraction = len(denovo_combined_snv.loc[(denovo_combined_snv['tri_CD'] == 'DCD')]) / triplet_counts_mutregions_CD['DCD'] / denovo_n_genomes_snv
C10 = len(denovo_combined_snv.loc[((denovo_combined_snv['tri_CD'] == 'CCD') | (denovo_combined_snv['tri_CD'] == 'DCC'))]) / triplet_counts_mutregions_CD[['CCD', 'DCC']].sum() / denovo_n_genomes_snv

denovo_mut_freq_CD = pd.Series([Cfission, Ccontraction, C10, Cfusion, Cexpansion, C01], index = ['Afission', 'Acontraction', 'A10', 'Afusion', 'Aexpansion', 'A01'])
denovo_mut_freq_CD.to_pickle('denovo/denovo_mut_freq_CD_remake.pickle')

In [ ]:
denovo_mut_freq_CD = pd.read_pickle('denovo/denovo_mut_freq_CD_remake.pickle')

#### Poisson random sampling for substitution rates

In [ ]:
denovo_mut_freq_AB_poisson = dict()
for i in range(200):
    Afusion = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & (denovo_combined_snv['tri_AB'] == 'ABA')])) / triplet_counts_mutregions_AB['ABA'] / denovo_n_genomes_snv
    Aexpansion = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & (denovo_combined_snv['tri_AB'] == 'BBB')])) / triplet_counts_mutregions_AB['BBB'] / denovo_n_genomes_snv
    A01 = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'A') & ((denovo_combined_snv['tri_AB'] == 'ABB') | (denovo_combined_snv['tri_AB'] == 'BBA'))])) / triplet_counts_mutregions_AB[['ABB', 'BBA']].sum() / denovo_n_genomes_snv
    Afission = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['tri_AB'] == 'AAA')])) / triplet_counts_mutregions_AB['AAA'] / denovo_n_genomes_snv
    Acontraction = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['tri_AB'] == 'BAB')])) / triplet_counts_mutregions_AB['BAB'] / denovo_n_genomes_snv
    A10 = np.random.poisson(len(denovo_combined_snv.loc[((denovo_combined_snv['tri_AB'] == 'AAB') | (denovo_combined_snv['tri_AB'] == 'BAA'))])) / triplet_counts_mutregions_AB[['AAB', 'BAA']].sum() / denovo_n_genomes_snv
    
    denovo_mut_freq_AB_poisson[i] = pd.Series([Afission, Acontraction, A10, Afusion, Aexpansion, A01], index = ['Afission', 'Acontraction', 'A10', 'Afusion', 'Aexpansion', 'A01'])
denovo_mut_freq_AB_poisson = pd.concat(denovo_mut_freq_AB_poisson, axis=1)
denovo_mut_freq_AB_poisson.to_pickle('denovo/denovo_mut_freq_AB_poisson_remake.pickle')

In [ ]:
denovo_mut_freq_AB_poisson = pd.read_pickle('denovo/denovo_mut_freq_AB_poisson_remake.pickle')

In [ ]:
denovo_mut_freq_CD_poisson = dict()
for i in range(200):
    Cfusion = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & (denovo_combined_snv['tri_CD'] == 'CDC')])) / triplet_counts_mutregions_CD['CDC'] / denovo_n_genomes_snv
    Cexpansion = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & (denovo_combined_snv['tri_CD'] == 'DDD')])) / triplet_counts_mutregions_CD['DDD'] / denovo_n_genomes_snv
    C01 = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['alt'] == 'C') & ((denovo_combined_snv['tri_CD'] == 'CDD') | (denovo_combined_snv['tri_CD'] == 'DDC'))])) / triplet_counts_mutregions_CD[['CDD', 'DDC']].sum() / denovo_n_genomes_snv
    Cfission = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['tri_CD'] == 'CCC')])) / triplet_counts_mutregions_CD['CCC'] / denovo_n_genomes_snv
    Ccontraction = np.random.poisson(len(denovo_combined_snv.loc[(denovo_combined_snv['tri_CD'] == 'DCD')])) / triplet_counts_mutregions_CD['DCD'] / denovo_n_genomes_snv
    C10 = np.random.poisson(len(denovo_combined_snv.loc[((denovo_combined_snv['tri_CD'] == 'CCD') | (denovo_combined_snv['tri_CD'] == 'DCC'))])) / triplet_counts_mutregions_CD[['CCD', 'DCC']].sum() / denovo_n_genomes_snv
    
    denovo_mut_freq_CD_poisson[i] = pd.Series([Cfission, Ccontraction, C10, Cfusion, Cexpansion, C01], index = ['Afission', 'Acontraction', 'A10', 'Afusion', 'Aexpansion', 'A01'])
denovo_mut_freq_CD_poisson = pd.concat(denovo_mut_freq_CD_poisson, axis=1)
denovo_mut_freq_CD_poisson.to_pickle('denovo/denovo_mut_freq_CD_poisson_remake.pickle')

In [ ]:
denovo_mut_freq_CD_poisson = pd.read_pickle('denovo/denovo_mut_freq_CD_poisson_remake.pickle')

## Calculate de novo indel rates

#### denominator for repeat lengths (count repeats in sequenceable regions)

In [ ]:
def make_dist_1_6(seq, name, out, shuffle = False):
    find_multi = r'([ATGC]{1,6}?)\1+'; find_multi = re.compile(find_multi)
    find_ind = r'([ATGC]{1,6}?)\1{1,}'; find_ind = re.compile(find_ind)
    find_L1 = dict()
    find_L1[1] = r'([ATGC]{1})\1{0}(?!\1)'; find_L1[1] = re.compile(find_L1[1])
    find_L1[2] = r'([ATGC]{2})\1{0}(?!\1)'; find_L1[2] = re.compile(find_L1[2])
    find_L1[3] = r'([ATGC]{3})\1{0}(?!\1)'; find_L1[3] = re.compile(find_L1[3])
    find_L1[4] = r'([ATGC]{4})\1{0}(?!\1)'; find_L1[4] = re.compile(find_L1[4])
    find_L1[5] = r'([ATGC]{5})\1{0}(?!\1)'; find_L1[5] = re.compile(find_L1[5])
    find_L1[6] = r'([ATGC]{6})\1{0}(?!\1)'; find_L1[6] = re.compile(find_L1[6])
    
    def count_L1(unit, seq, out):
        out[unit] = pd.Series([m.group() for m in re.finditer(find_L1[unit], seq)]).value_counts()

    seq = seq.upper()
    if shuffle == True:
        seq = ''.join(random.sample(seq, len(seq)))
    
    L1 = dict()
    for unit in range(1,7):
        count_L1(unit, seq, L1)
    L1 = pd.concat(L1).reset_index()
    L1.columns = ['unit_len', 'motif', 'count']
    L1['length'] = 1
    multi = pd.Series([m.group() for m in re.finditer(find_multi, seq, overlapped=True)]).value_counts().reset_index()
    if len(multi) >0:
        multi['motif'] = [re.split(find_ind, ind)[1] for ind in multi['index']]
        multi['unit_len'] = multi['motif'].str.len()
        multi['length'] = [index.count(motif) for index, motif in zip(multi['index'], multi['motif'])]
    
    counts = pd.concat([L1, multi])
    counts['motif_RC'] = [repeat_frames_RC(motif)[0] for motif in counts['motif']]
    counts = counts.groupby(['unit_len', 'motif_RC', 'length'])['count'].sum().unstack().transpose().sort_index(axis=0).fillna(0).astype(int)
    counts  = counts.sub(counts.shift(-1), fill_value = 0)

    out[name] = counts

In [ ]:
# A counts for motif length 1-6 in sequenceable regions of HG38   
genome_counts = manager.dict()
job = [Process(target=make_dist_1_6, args=(hg38_genome[record[0]][int(record[1]):int(record[1]+1e5)], record, genome_counts)) for record in count_by_100kb.index]
_ = [p.start() for p in job]
_ = [p.join() for p in job]
_ = [p.close() for p in job]

In [ ]:
genome_counts_all = pd.concat(dict(genome_counts), axis=1).T.groupby(['unit_len', 'motif_RC']).sum().T.astype(int)

In [ ]:
genome_counts_all.to_pickle('denovo/repeat_counts_mutregions_A_remake.pickle')

In [ ]:
# need B[1] and B[2:] counts for B>A rates
# must be run per motif
def make_B_dist(seq, motif, out, name):
    B_dist = dict()
    for frame in repeat_frames_RC(motif):
        find_B = '(?:(?!' + frame + ')[ATGC])+'
        current = pd.DataFrame([m.span() for m in re.finditer(find_B, seq)])
        if len(current) >0:
            current = current[1] - current[0] - (len(motif)-1)
            B_dist[frame] = current.value_counts()
    if len(B_dist) > 0:
        B_dist = pd.concat(B_dist, axis=1).fillna(0).sort_index().sum(axis=1)
    out[name] = B_dist
repeats_1_3 = ['A', 'C', 'AC', 'AT', 'AG', 'CG', 'AAT', 'AAG', 'AAC', 'ATC', 'ACT', 'AGG', 'AGC', 'ACG', 'ACC', 'CCG']

In [ ]:
B_genome_counts = dict()
counter = 0
for motif in repeats_1_3:
# B counts for motif length 1-3 in sequenceable regions of HG38   
    genome_counts_B = manager.dict()
    job = [Process(target=make_B_dist, args=(hg38_genome[record[0]][int(record[1]):int(record[1]+1e5)], motif, genome_counts_B, str(record))) for record in count_by_100kb.index]
    _ = [p.start() for p in job]
    _ = [p.join() for p in job]
    _ = [p.close() for p in job]
    B_genome_counts[motif] = pd.concat(dict(genome_counts_B), axis=1).sum(axis=1).sort_index()
    counter +=1; print('\r' + str(counter) + '/' + str(len(repeats_1_3)), end = '      ')

In [ ]:
B_genome_counts_all = pd.concat(B_genome_counts, axis=1).fillna(0).astype(int)
B_genome_counts_all.to_pickle('denovo/repeat_counts_mutregions_B_remake.pickle')

In [ ]:
repeat_counts_mutregions_A_RC = pd.read_pickle('denovo/repeat_counts_mutregions_A_remake.pickle').droplevel(0, axis=1)
repeat_counts_mutregions_B_RC = pd.read_pickle('denovo/repeat_counts_mutregions_B_remake.pickle')

#### indel counts

In [ ]:
denovo_combined_indel = pd.read_csv('denovo/all_denovo_indels.csv')
denovo_combined_indel['chrom'] = [int(chrom) if chrom not in ['X', 'Y'] else chrom for chrom in denovo_combined_indel['chrom']]
denovo_combined_indel['REF+100'] = [get_sequence(chrom, pos, pos+101, hg38_genome) for chrom, pos in zip(denovo_combined_indel['chrom'], denovo_combined_indel['pos'])]
denovo_combined_indel['REF-100'] = [get_sequence(chrom, pos-101, pos, hg38_genome) for chrom, pos in zip(denovo_combined_indel['chrom'], denovo_combined_indel['pos'])]
denovo_combined_indel['indel'] = (denovo_combined_indel['ref'].str.len() > 1).replace(False, 'ins').replace(True, 'del')
denovo_combined_indel['indel_seq'] = denovo_combined_indel['alt'].str[1:] + denovo_combined_indel['ref'].str[1:]
denovo_combined_indel['indel_len'] = denovo_combined_indel['indel_seq'].str.len()

for unit_len in range(1,7):
    denovo_combined_indel['down_'+str(unit_len)] = denovo_combined_indel['REF+100'].str[:unit_len]
    denovo_combined_indel['up_'+str(unit_len)] = denovo_combined_indel['REF-100'].str[-unit_len:]
    denovo_combined_indel['repeat_seq_down_'+str(unit_len)] = [re.search('(?:' + motif + ')+', ref)[0] for ref, motif in zip(denovo_combined_indel['REF+100'], denovo_combined_indel['down_'+str(unit_len)])]
    denovo_combined_indel['repeat_seq_up_'+str(unit_len)] = [re.search('(?:' + motif[::-1] + ')+', ref[::-1])[0] for ref, motif in zip(denovo_combined_indel['REF-100'], denovo_combined_indel['up_'+str(unit_len)])]
    denovo_combined_indel['repeat_units_up_'+str(unit_len)] = denovo_combined_indel['repeat_seq_up_'+str(unit_len)].str.len() / unit_len
    denovo_combined_indel['repeat_units_down_'+str(unit_len)] = denovo_combined_indel['repeat_seq_down_'+str(unit_len)].str.len() / unit_len
    denovo_combined_indel['same_bothsides_'+str(unit_len)] = denovo_combined_indel['up_'+str(unit_len)] == denovo_combined_indel['down_'+str(unit_len)]
    
    # for indels in the middle of a repeat, put the total count in the 'up' category and remove the 'down' count
    denovo_combined_indel['repeat_units_up_'+str(unit_len)] += (denovo_combined_indel['repeat_units_down_'+str(unit_len)] * denovo_combined_indel['same_bothsides_'+str(unit_len)])
    denovo_combined_indel['repeat_units_down_'+str(unit_len)] -= (denovo_combined_indel['repeat_units_down_'+str(unit_len)] * denovo_combined_indel['same_bothsides_'+str(unit_len)])
    denovo_combined_indel['repeat_units_down_'+str(unit_len)] = denovo_combined_indel['repeat_units_down_'+str(unit_len)].replace(0, np.nan)
    
    denovo_combined_indel['indel_is_rep_up_'+str(unit_len)] = [''.join(rep * int(indel_len / unit_len)) == indel_seq for rep, indel_len, indel_seq in zip(denovo_combined_indel['up_'+str(unit_len)], denovo_combined_indel['indel_len'], denovo_combined_indel['indel_seq'])]
    denovo_combined_indel['indel_is_rep_down_'+str(unit_len)] = [''.join(rep * int(indel_len / unit_len)) == indel_seq for rep, indel_len, indel_seq in zip(denovo_combined_indel['down_'+str(unit_len)], denovo_combined_indel['indel_len'], denovo_combined_indel['indel_seq'])]

In [ ]:
exp_counts = dict(); con_counts = dict(); nonexp_counts = dict()
for unit_len in range(1,7):
    # A>AA
    exp_up = denovo_combined_indel.groupby(['indel', 'indel_is_rep_up_'+str(unit_len), 'indel_len', 'repeat_units_up_'+str(unit_len), 'up_'+str(unit_len)])['chrom'].count()['ins'].reindex([True], level = 0).unstack().droplevel(0)
    exp_down = denovo_combined_indel.groupby(['indel', 'indel_is_rep_down_'+str(unit_len), 'indel_len', 'repeat_units_down_'+str(unit_len), 'down_'+str(unit_len)])['chrom'].count()['ins'].reindex([True], level = 0).unstack().droplevel(0)
    exp_up.index.names = ['indel_len', 'repeat_units']; exp_down.index.names = ['indel_len', 'repeat_units']
    exp_counts[unit_len] = exp_up.add(exp_down, fill_value = 0)
    #AA>A
    con_up = denovo_combined_indel.groupby(['indel', 'indel_is_rep_up_'+str(unit_len), 'indel_len', 'repeat_units_up_'+str(unit_len), 'up_'+str(unit_len)])['chrom'].count()['del'].reindex([True], level = 0).unstack().droplevel(0)
    con_down = denovo_combined_indel.groupby(['indel', 'indel_is_rep_down_'+str(unit_len), 'indel_len', 'repeat_units_down_'+str(unit_len), 'down_'+str(unit_len)])['chrom'].count()['del'].reindex([True], level = 0).unstack().droplevel(0)
    con_up.index.names = ['indel_len', 'repeat_units']; con_down.index.names = ['indel_len', 'repeat_units']
    con_counts[unit_len] = con_up.add(con_down, fill_value = 0)
    # AA>ABA interruptions (requires A on both sides, length is sum of both sides)
    nonexp_counts[unit_len] = denovo_combined_indel.groupby(['indel', 'same_bothsides_'+str(unit_len), 'indel_is_rep_up_'+str(unit_len), 'indel_len', 'repeat_units_up_'+str(unit_len), 'up_'+str(unit_len)])['chrom'].count()['ins'].reindex([True], level = 0).reindex([False], level = 1).unstack().droplevel(0).droplevel(0)
    nonexp_counts[unit_len].index.names = ['indel_len', 'repeat_units']
exp_counts = pd.concat(exp_counts, axis=1).droplevel(0, axis=1); con_counts = pd.concat(con_counts, axis=1).droplevel(0, axis=1); nonexp_counts = pd.concat(nonexp_counts, axis=1).droplevel(0, axis=1)

#BB>BAB (0-class for expansions)
B_exp = dict()
for unit_len in range(1,7):
    B_exp[unit_len] = denovo_combined_indel.groupby(['indel', 'indel_is_rep_up_'+str(unit_len), 'indel_is_rep_down_'+str(unit_len), 'indel_len', 'indel_seq'])['chrom'].count()['ins'].reindex([False], level = 0).reindex([False], level = 1).droplevel(0).droplevel(0).unstack().reindex([motif for motif in exp_counts.columns if len(motif) == unit_len], axis=1)
    B_exp[unit_len]['repeat_units'] = 0
    B_exp[unit_len].set_index('repeat_units', append = True, inplace = True)
B_exp = pd.concat(B_exp, axis=1).droplevel(0, axis=1)
exp_counts = pd.concat([exp_counts, B_exp]).sort_index()

# BB>B deletions (0-class for contractions)
B_del = dict()
B_del_counts = denovo_combined_indel.groupby(['indel', 'indel_len', 'indel_seq'])['chrom'].count()['del'].unstack().reindex([col for col in con_counts.columns], axis=1)
for motif in con_counts.columns:
    B_del[motif] = B_del_counts.reindex([col for col in con_counts.columns if (len(col) == len(motif)) & (col != motif)], axis=1).sum(axis=1)
    B_del[motif] = pd.DataFrame(B_del[motif], columns = [motif])
    B_del[motif]['repeat_units'] = 0
    B_del[motif].set_index('repeat_units', append = True, inplace = True)
B_del = pd.concat(B_del, axis=1).droplevel(0, axis=1)
con_counts = pd.concat([con_counts, B_del]).sort_index()

# B>BB insertions (length-independent) (0-class for non-expansions)
B_ins = dict()
B_ins_counts = denovo_combined_indel.groupby(['indel', 'indel_len', 'indel_seq'])['chrom'].count()['ins'].unstack().reindex([col for col in exp_counts.columns], axis=1)
for motif in exp_counts.columns:
    B_ins[motif] = B_ins_counts.reindex([col for col in exp_counts.columns if (len(col) == len(motif)) & (col != motif)], axis=1).sum(axis=1)
    B_ins[motif] = pd.DataFrame(B_ins[motif], columns = [motif])
    B_ins[motif]['repeat_units'] = 0
    B_ins[motif].set_index('repeat_units', append = True, inplace = True)
B_ins = pd.concat(B_ins, axis=1).droplevel(0, axis=1)
nonexp_counts = pd.concat([nonexp_counts, B_ins]).sort_index()

In [ ]:
exp_counts_std = dict(); con_counts_std = dict(); nonexp_counts_std = dict()
for repeat in exp_counts:
    if repeat_frames_RC(repeat)[0] not in exp_counts_std.keys():
        exp_counts_std[repeat_frames_RC(repeat)[0]] = exp_counts.reindex(repeat_frames_RC(repeat), axis=1).sum(axis=1)
        con_counts_std[repeat_frames_RC(repeat)[0]] = con_counts.reindex(repeat_frames_RC(repeat), axis=1).sum(axis=1)
        nonexp_counts_std[repeat_frames_RC(repeat)[0]] = nonexp_counts.reindex(repeat_frames_RC(repeat), axis=1).sum(axis=1)
exp_counts_std = pd.concat(exp_counts_std, axis=1); con_counts_std = pd.concat(con_counts_std, axis=1); nonexp_counts_std = pd.concat(nonexp_counts_std, axis=1)

In [ ]:
exp_counts_std.to_pickle('denovo/denovo_combined_expansion_counts_all_remake.pickle')
con_counts_std.to_pickle('denovo/denovo_combined_contraction_deletion_counts_all_remake.pickle')
nonexp_counts_std.to_pickle('denovo/denovo_combined_nonexpansion_insertion_counts_all_remake.pickle')

In [ ]:
exp_counts_std = pd.read_pickle('denovo/denovo_combined_expansion_counts_all_remake.pickle')
con_counts_std = pd.read_pickle('denovo/denovo_combined_contraction_deletion_counts_all_remake.pickle')
nonexp_counts_std = pd.read_pickle('denovo/denovo_combined_nonexpansion_insertion_counts_all_remake.pickle')

In [ ]:
exp_counts_inframe = dict(); con_counts_inframe = dict(); nonexp_counts_inframe = dict()
for motif in exp_counts_std:
    exp_counts_inframe[motif] = exp_counts_std[motif].loc[len(motif)]
    con_counts_inframe[motif] = con_counts_std[motif].loc[len(motif)]
    nonexp_counts_inframe[motif] = nonexp_counts_std[motif].loc[len(motif)]
exp_counts_inframe = pd.concat(exp_counts_inframe, axis=1)
con_counts_inframe = pd.concat(con_counts_inframe, axis=1)
nonexp_counts_inframe = pd.concat(nonexp_counts_inframe, axis=1)

In [ ]:
exp_counts_inframe.to_pickle('denovo/denovo_combined_expansion_counts_remake.pickle')
con_counts_inframe.to_pickle('denovo/denovo_combined_contraction_deletion_counts_remake.pickle')
nonexp_counts_inframe.to_pickle('denovo/denovo_combined_nonexpansion_insertion_counts_remake.pickle')

In [ ]:
exp_counts_inframe = pd.read_pickle('denovo/denovo_combined_expansion_counts_remake.pickle')
con_counts_inframe = pd.read_pickle('denovo/denovo_combined_contraction_deletion_counts_remake.pickle')
nonexp_counts_inframe = pd.read_pickle('denovo/denovo_combined_nonexpansion_insertion_counts_remake.pickle')

#### calculate rates

In [ ]:
repeat_counts_mutregions_totalbases = dict()
repeat_counts_mutregions_totalbases['A'] = repeat_counts_mutregions_A_RC.mul(repeat_counts_mutregions_A_RC.index, axis=0) # rate per repeat unit
repeat_counts_mutregions_totalbases['B'] = repeat_counts_mutregions_B_RC.mul(repeat_counts_mutregions_B_RC.index, axis=0)
repeat_counts_mutregions_forexp = repeat_counts_mutregions_totalbases['A']
repeat_counts_mutregions_forcon = repeat_counts_mutregions_totalbases['A']
# BB>BAB, denominator is number of B bases excluding single Bs 
repeat_counts_mutregions_forexp.loc[0] = repeat_counts_mutregions_totalbases['B'][2:].sum()
# B>BB insertions, treating as length-independent, denominator is total number of B bases
# B>_ deletions, treating as length-independent, denominator is total number of B bases, includes fusion events (subtract later if distinguishing)
repeat_counts_mutregions_forcon.loc[0] = repeat_counts_mutregions_totalbases['B'].sum()

repeat_counts_mutregions_forexp = repeat_counts_mutregions_forexp.sort_index()
repeat_counts_mutregions_forcon = repeat_counts_mutregions_forcon.sort_index()

In [ ]:
denovo_exp_rate = (exp_counts_inframe.div(repeat_counts_mutregions_forexp.reindex(exp_counts_inframe.columns, axis=1)) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
denovo_con_rate = (con_counts_inframe.div(repeat_counts_mutregions_forcon.reindex(con_counts_inframe.columns, axis=1)) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
denovo_nonexp_rate = (nonexp_counts_inframe.div(repeat_counts_mutregions_forcon.reindex(nonexp_counts_inframe.columns, axis=1)) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)

In [ ]:
for unit_len in [1,2,3,4]:
    current_reps = [rep for rep in denovo_exp_rate.columns if len(rep) == unit_len]
    denovo_exp_rate['unit_'+str(unit_len)] = (exp_counts_inframe[current_reps].sum(axis=1) / repeat_counts_mutregions_forexp[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
    denovo_con_rate['unit_'+str(unit_len)] = (con_counts_inframe[current_reps].sum(axis=1) / repeat_counts_mutregions_forcon[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
    denovo_nonexp_rate['unit_'+str(unit_len)] = (nonexp_counts_inframe[current_reps].sum(axis=1) / repeat_counts_mutregions_forcon[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)

In [ ]:
denovo_exp_rate.to_pickle('denovo/denovo_exp_rate_remake.pickle')
denovo_con_rate.to_pickle('denovo/denovo_con_rate_remake.pickle')
denovo_nonexp_rate.to_pickle('denovo/denovo_nonexp_rate_remake.pickle')

In [ ]:
denovo_exp_rate = pd.read_pickle('denovo/denovo_exp_rate_remake.pickle')
denovo_con_rate = pd.read_pickle('denovo/denovo_con_rate_remake.pickle')
denovo_nonexp_rate = pd.read_pickle('denovo/denovo_nonexp_rate_remake.pickle')

#### Poisson random sampling for indel rates

In [ ]:
denovo_exp_rate_poisson = dict()
denovo_con_rate_poisson = dict()
denovo_nonexp_rate_poisson = dict()
for i in range(200):
    denovo_exp_rate_poisson[i] = (exp_counts_inframe.fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forexp.reindex(exp_counts_inframe.columns, axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
    denovo_con_rate_poisson[i] = (con_counts_inframe.fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forcon.reindex(con_counts_inframe.columns, axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
    denovo_nonexp_rate_poisson[i] = (nonexp_counts_inframe.fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forcon.reindex(nonexp_counts_inframe.columns, axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)

denovo_exp_rate_poisson = pd.concat(denovo_exp_rate_poisson)
denovo_con_rate_poisson = pd.concat(denovo_con_rate_poisson)
denovo_nonexp_rate_poisson = pd.concat(denovo_nonexp_rate_poisson)

In [ ]:
denovo_exp_rate_poisson_units = dict()
denovo_con_rate_poisson_units = dict()
denovo_nonexp_rate_poisson_units = dict()
for unit_len in [1,2,3,4]:
    for i in range(200):
        current_reps = [rep for rep in denovo_exp_rate.columns if len(rep) == unit_len]
        denovo_exp_rate_poisson_units[i] = (exp_counts_inframe[current_reps].sum(axis=1).fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forexp[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
        denovo_con_rate_poisson_units[i] = (con_counts_inframe[current_reps].sum(axis=1).fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forcon[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
        denovo_nonexp_rate_poisson_units[i] = (nonexp_counts_inframe[current_reps].sum(axis=1).fillna(0).apply(lambda x: np.random.poisson(x)) / repeat_counts_mutregions_forcon[current_reps].sum(axis=1) / denovo_n_genomes_indel).dropna(how = 'all', axis=0)
    denovo_exp_rate_poisson['unit_'+str(unit_len)] = pd.concat(denovo_exp_rate_poisson_units, axis=0).replace(0, np.nan)
    denovo_con_rate_poisson['unit_'+str(unit_len)] = pd.concat(denovo_con_rate_poisson_units, axis=0).replace(0, np.nan)
    denovo_nonexp_rate_poisson['unit_'+str(unit_len)] = pd.concat(denovo_nonexp_rate_poisson_units, axis=0).replace(0, np.nan)

In [ ]:
denovo_exp_rate_poisson.to_pickle('denovo/denovo_exp_rate_poisson_remake.pickle')
denovo_con_rate_poisson.to_pickle('denovo/denovo_con_rate_poisson_remake.pickle')
denovo_nonexp_rate_poisson.to_pickle('denovo/denovo_nonexp_rate_poisson_remake.pickle')

In [ ]:
denovo_exp_rate_poisson = pd.read_pickle('denovo/denovo_exp_rate_poisson_remake.pickle')
denovo_con_rate_poisson = pd.read_pickle('denovo/denovo_con_rate_poisson_remake.pickle')
denovo_nonexp_rate_poisson = pd.read_pickle('denovo/denovo_nonexp_rate_poisson_remake.pickle')

### Insertion/deletion size per event

In [ ]:
denovo_combined_indel['indel_len_dir'] = [nt if indel == 'ins' else -nt for nt, indel in zip(denovo_combined_indel['indel_len'], denovo_combined_indel['indel'])]

In [ ]:
unit_fraction_of_events_per_length = dict()
unit_fraction_of_events_per_length[1] = denovo_combined_indel.groupby(['repeat_units_down_1', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).div(denovo_combined_indel.groupby(['repeat_units_down_1', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).sum(axis=1), axis=0) 
unit_fraction_of_events_per_length[2] = denovo_combined_indel.groupby(['repeat_units_down_2', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).div(denovo_combined_indel.groupby(['repeat_units_down_2', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).sum(axis=1), axis=0)
unit_fraction_of_events_per_length[3] = denovo_combined_indel.groupby(['repeat_units_down_3', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).div(denovo_combined_indel.groupby(['repeat_units_down_3', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).sum(axis=1), axis=0)
unit_fraction_of_events_per_length[4] = denovo_combined_indel.groupby(['repeat_units_down_4', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).div(denovo_combined_indel.groupby(['repeat_units_down_4', 'indel_len_dir'])['chrom'].count().unstack().fillna(0).sum(axis=1), axis=0)

In [ ]:
# Fig. S4 (top)
fig = make_subplots(rows = 4, cols = 1, vertical_spacing = 0.015, shared_xaxes = True, row_titles = ['unit=1', 'unit=2', 'unit=3', 'unit=4'], x_title = 'repeat tract length (units)', y_title = 'fraction of events per length bin')
for unit_len in range(1,5):
    for inslen in [1,2,3,4]:
        fig.add_trace(go.Bar(x = unit_fraction_of_events_per_length[unit_len].index, y = unit_fraction_of_events_per_length[unit_len][inslen], name = str(np.abs(inslen)), legendgroup = str(np.abs(inslen)), showlegend = True if unit_len == 1 else False), row = unit_len, col = 1)
    fig.add_trace(go.Bar(x = unit_fraction_of_events_per_length[unit_len].index, y = unit_fraction_of_events_per_length[unit_len][[col for col in unit_fraction_of_events_per_length[unit_len].columns if col > 4]].sum(axis=1), name = '>4', legendgroup = 'other', showlegend = True if unit_len == 1 else False), row = unit_len, col = 1)
    for inslen in [-1,-2,-3,-4]:
        fig.add_trace(go.Bar(x = unit_fraction_of_events_per_length[unit_len].index, y = -unit_fraction_of_events_per_length[unit_len][inslen], name = str(np.abs(inslen)), legendgroup = str(np.abs(inslen)), showlegend = False), row = unit_len, col = 1)
    fig.add_trace(go.Bar(x = unit_fraction_of_events_per_length[unit_len].index, y = - unit_fraction_of_events_per_length[unit_len][[col for col in unit_fraction_of_events_per_length[unit_len].columns if col < -4]].sum(axis=1), name = '>4', legendgroup = 'other', showlegend = False), row = unit_len, col = 1)
    fig.update_xaxes(range = [0.5,10.5], dtick = 1, row = unit_len, col = 1)
fig.update_yaxes(range = [-1,1], tickvals = [-0.5, 0, 0.5])
fig.update_layout(font=dict(family = 'Arial', size = 14), height = 400, width = 600, title = 'de novo', colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:5], legend_title_text='nt added', margin={'t':30,'l':80,'b':60,'r':20})
fig.update_layout(barmode = 'relative')
fig.show()

In [ ]:
fig.write_image('plots/figS4a.pdf')

## Calculate substitution context rates for trinucleotide repeats

#### Count mutations, checking each triplet frame for repeat status

In [ ]:
def check_tri_context(seq):
    trisplit = re.findall('.'*3, seq)
    if trisplit[0] == trisplit[1]:
        if trisplit[1] == trisplit[2]:
            return 'AAA', repeat_frames_RC(trisplit[1])[0]
        else:
            return 'AAB', repeat_frames_RC(trisplit[1])[0]
    else:
        if trisplit[1] == trisplit[2]:
            return 'BAA', repeat_frames_RC(trisplit[1])[0]
        else:
            return 'BAB', repeat_frames_RC(trisplit[1])[0]

In [ ]:
# 9-mer context in three frames -> 11nt context needed
denovo_combined_snv['11nt'] = [get_sequence(chrom, pos-6, pos+5, hg38_genome) for chrom, pos in zip(denovo_combined_snv['chrom'], denovo_combined_snv['pos'])]
denovo_combined_snv['11nt_AB_1'] = [check_tri_context(seq) for seq in denovo_combined_snv['11nt']]
denovo_combined_snv['11nt_AB_2'] = [check_tri_context(seq[1:]) for seq in denovo_combined_snv['11nt']]
denovo_combined_snv['11nt_AB_3'] = [check_tri_context(seq[2:]) for seq in denovo_combined_snv['11nt']]

denovo_triplet_context_count = denovo_combined_snv.groupby(['11nt_AB_1'])['pos'].count().add(denovo_combined_snv.groupby(['11nt_AB_2'])['pos'].count(), fill_value = 0).add(denovo_combined_snv.groupby(['11nt_AB_3'])['pos'].count(), fill_value = 0)
denovo_triplet_context_count.index = pd.MultiIndex.from_tuples(denovo_triplet_context_count.index)
denovo_triplet_context_count = denovo_triplet_context_count.unstack().transpose()
denovo_triplet_context_count['AAB'] = denovo_triplet_context_count['AAB'] + denovo_triplet_context_count['BAA']
del denovo_triplet_context_count['BAA']
denovo_triplet_context_count.columns = ['Afission', 'Acontraction', 'A10']

In [ ]:
denovo_combined_snv['11nt_alt'] = [el[0:5]+alt+el[6:] for el, alt in zip(denovo_combined_snv['11nt'], denovo_combined_snv['alt'])]
denovo_combined_snv['11nt_alt_AB_1'] = [check_tri_context(seq) for seq in denovo_combined_snv['11nt_alt']]
denovo_combined_snv['11nt_alt_AB_2'] = [check_tri_context(seq[1:]) for seq in denovo_combined_snv['11nt_alt']]
denovo_combined_snv['11nt_alt_AB_3'] = [check_tri_context(seq[2:]) for seq in denovo_combined_snv['11nt_alt']]

denovo_triplet_context_count_alt = denovo_combined_snv.groupby(['11nt_alt_AB_1'])['pos'].count().add(denovo_combined_snv.groupby(['11nt_alt_AB_2'])['pos'].count(), fill_value = 0).add(denovo_combined_snv.groupby(['11nt_alt_AB_3'])['pos'].count(), fill_value = 0)
denovo_triplet_context_count_alt.index = pd.MultiIndex.from_tuples(denovo_triplet_context_count_alt.index)
denovo_triplet_context_count_alt = denovo_triplet_context_count_alt.unstack().transpose()
denovo_triplet_context_count_alt['AAB'] = denovo_triplet_context_count_alt['AAB'] + denovo_triplet_context_count_alt['BAA']
del denovo_triplet_context_count_alt['BAA']
denovo_triplet_context_count_alt.columns = ['Afusion', 'Aexpansion', 'A01']

In [ ]:
denovo_triplet_context_count_all = pd.concat([denovo_triplet_context_count, denovo_triplet_context_count_alt], axis=1)

In [ ]:
denovo_triplet_context_count_all.to_pickle('denovo/denovo_triplet_context_count_all_remake.pickle')

In [ ]:
denovo_triplet_context_count_all = pd.read_pickle('denovo/denovo_triplet_context_count_all_remake.pickle')

#### Count all 9-mer contexts in the sequenceable genome

In [ ]:
nine_counts_mutregions = dict()
for chrom, pos in count_by_100kb.index:
    seq = hg38_genome[chrom][int(pos):int(pos+1e5)]
    nine_counts_mutregions[(chrom, pos)] = pd.Series(re.findall('.........', seq) + re.findall('.........', seq[1:]) + re.findall('.........', seq[2:]) + re.findall('.........', seq[3:]) + re.findall('.........', seq[4:]) + re.findall('.........', seq[5:]) + re.findall('.........', seq[6:]) + re.findall('.........', seq[7:]) + re.findall('.........', seq[8:]), dtype = 'object').value_counts()
    print ('\r' + 'finished chr'+str(chrom) + ' '+str(pos), end = ' ')

In [ ]:
nine_counts_mutregions_sum = pd.Series(dtype = 'int64')
counter = 0
for part in list(nine_counts_mutregions.keys()):
    nine_counts_mutregions_sum = nine_counts_mutregions_sum.add(nine_counts_mutregions[part], fill_value = 0)
    counter +=1; print('\r' + str(counter), end = ' ')
nine_counts_mutregions_sum.to_pickle('denovo/ninelet_totals_hg38_mutregions_remake.pickle')

In [ ]:
nine_counts_mutregions_sum = pd.read_pickle('denovo/ninelet_totals_hg38_mutregions_remake.pickle')

In [ ]:
nine_counts_mutregions_sum_triindex = nine_counts_mutregions_sum.copy()
nine_counts_mutregions_sum_triindex.index = pd.MultiIndex.from_tuples([re.findall('...', seq) for seq in nine_counts_mutregions_sum.index])
nine_counts_mutregions_sum_triindex = nine_counts_mutregions_sum_triindex.reset_index()

In [ ]:
nine_counts_triplet_context_count_all = pd.DataFrame(0, index = denovo_triplet_context_count_all.index, columns = denovo_triplet_context_count_all.columns)
for repeat in nine_counts_triplet_context_count_all.index:
    for motif in repeat_frames_RC(repeat):
        nine_counts_triplet_context_count_all.loc[repeat, 'Afission'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] == motif) & (nine_counts_mutregions_sum_triindex['level_1'] == motif) & (nine_counts_mutregions_sum_triindex['level_2'] == motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'Acontraction'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] == motif) & (nine_counts_mutregions_sum_triindex['level_1'] == motif) & (nine_counts_mutregions_sum_triindex['level_2'] != motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'Acontraction'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] != motif) & (nine_counts_mutregions_sum_triindex['level_1'] == motif) & (nine_counts_mutregions_sum_triindex['level_2'] == motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'A10'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] != motif) & (nine_counts_mutregions_sum_triindex['level_1'] == motif) & (nine_counts_mutregions_sum_triindex['level_2'] != motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'Afusion'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] == motif) & (nine_counts_mutregions_sum_triindex['level_1'] != motif) & (nine_counts_mutregions_sum_triindex['level_2'] == motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'Aexpansion'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] == motif) & (nine_counts_mutregions_sum_triindex['level_1'] != motif) & (nine_counts_mutregions_sum_triindex['level_2'] != motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'Aexpansion'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] != motif) & (nine_counts_mutregions_sum_triindex['level_1'] != motif) & (nine_counts_mutregions_sum_triindex['level_2'] == motif)][0].sum()
        nine_counts_triplet_context_count_all.loc[repeat, 'A01'] += nine_counts_mutregions_sum_triindex.loc[(nine_counts_mutregions_sum_triindex['level_0'] != motif) & (nine_counts_mutregions_sum_triindex['level_1'] != motif) & (nine_counts_mutregions_sum_triindex['level_2'] != motif)][0].sum()

#### Calculate rates

In [ ]:
denovo_substitution_context_rate = denovo_triplet_context_count_all /3 / nine_counts_triplet_context_count_all / denovo_n_genomes_snv
#denovo_substitution_context_rate.loc['A'] = denovo_mut_freq_AB
#denovo_substitution_context_rate.loc['C'] = denovo_mut_freq_CD
denovo_substitution_context_rate.to_pickle('denovo/denovo_mut_freq_triplets_remake.pickle')

In [ ]:
denovo_substitution_context_rate = pd.read_pickle('denovo/denovo_mut_freq_triplets_remake.pickle')

In [ ]:
denovo_mut_freq_triplets_poisson = dict()
for i in range(200):
    denovo_mut_freq_triplets_poisson[i] = (denovo_triplet_context_count_all /3).apply(np.random.poisson) / nine_counts_triplet_context_count_all / denovo_n_genomes_snv
denovo_mut_freq_triplets_poisson = pd.concat(denovo_mut_freq_triplets_poisson)
denovo_mut_freq_triplets_poisson.to_pickle('denovo/denovo_mut_freq_triplets_poisson_remake.pickle')

In [ ]:
# put triplets and As into one file
denovo_mut_freq_triplets_poisson = pd.read_pickle('denovo/denovo_mut_freq_triplets_poisson_remake.pickle')
denovo_mut_freq_AB_poisson = pd.read_pickle('denovo/denovo_mut_freq_AB_poisson.pickle')
denovo_mut_freq_AB_poisson = denovo_mut_freq_AB_poisson.transpose()
denovo_mut_freq_AB_poisson.index = pd.MultiIndex.from_tuples([(i, 'A') for i in denovo_mut_freq_AB_poisson.index])
denovo_mut_freq_triplets_poisson = pd.concat([denovo_mut_freq_triplets_poisson, denovo_mut_freq_AB_poisson], axis=0).sort_index()
denovo_mut_freq_triplets_poisson.to_pickle('denovo/denovo_mut_freq_triplets_poisson_remake.pickle')

## Calculate substitution context rates for dinucleotide repeats

#### Count mutations, checking each triplet frame for repeat status

In [ ]:
def check_di_context(seq):
    displit = re.findall('.'*2, seq)
    if displit[0] == displit[1]:
        if displit[1] == displit[2]:
            return 'AAA', repeat_frames_RC(displit[1])[0]
        else:
            return 'AAB', repeat_frames_RC(displit[1])[0]
    else:
        if displit[1] == displit[2]:
            return 'BAA', repeat_frames_RC(displit[1])[0]
        else:
            return 'BAB', repeat_frames_RC(displit[1])[0]

In [ ]:
# 6-mer context in two frames -> 7nt context needed
denovo_combined_snv['7nt'] = [get_sequence(chrom, pos-4, pos+3, hg38_genome) for chrom, pos in zip(denovo_combined_snv['chrom'], denovo_combined_snv['pos'])]
denovo_combined_snv['7nt_AB_1'] = [check_di_context(seq) for seq in denovo_combined_snv['7nt']]
denovo_combined_snv['7nt_AB_2'] = [check_di_context(seq[1:]) for seq in denovo_combined_snv['7nt']]

denovo_dinuc_context_count = denovo_combined_snv.groupby(['7nt_AB_1'])['pos'].count().add(denovo_combined_snv.groupby(['7nt_AB_2'])['pos'].count(), fill_value = 0)
denovo_dinuc_context_count.index = pd.MultiIndex.from_tuples(denovo_dinuc_context_count.index)
denovo_dinuc_context_count = denovo_dinuc_context_count.unstack().transpose()
denovo_dinuc_context_count['AAB'] = denovo_dinuc_context_count['AAB'] + denovo_dinuc_context_count['BAA']
del denovo_dinuc_context_count['BAA']
denovo_dinuc_context_count.columns = ['Afission', 'Acontraction', 'A10']

In [ ]:
denovo_combined_snv['7nt_alt'] = [sev[0:4]+alt+sev[5:] for sev, alt in zip(denovo_combined_snv['7nt'], denovo_combined_snv['alt'])]
denovo_combined_snv['7nt_alt_AB_1'] = [check_di_context(seq) for seq in denovo_combined_snv['7nt_alt']]
denovo_combined_snv['7nt_alt_AB_2'] = [check_di_context(seq[1:]) for seq in denovo_combined_snv['7nt_alt']]

denovo_dinuc_context_count_alt = denovo_combined_snv.groupby(['7nt_alt_AB_1'])['pos'].count().add(denovo_combined_snv.groupby(['7nt_alt_AB_2'])['pos'].count(), fill_value = 0)
denovo_dinuc_context_count_alt.index = pd.MultiIndex.from_tuples(denovo_dinuc_context_count_alt.index)
denovo_dinuc_context_count_alt = denovo_dinuc_context_count_alt.unstack().transpose()
denovo_dinuc_context_count_alt['AAB'] = denovo_dinuc_context_count_alt['AAB'] + denovo_dinuc_context_count_alt['BAA']
del denovo_dinuc_context_count_alt['BAA']
denovo_dinuc_context_count_alt.columns = ['Afusion', 'Aexpansion', 'A01']

In [ ]:
denovo_dinuc_context_count_all = pd.concat([denovo_dinuc_context_count, denovo_dinuc_context_count_alt], axis=1)

In [ ]:
denovo_dinuc_context_count_all.to_pickle('denovo/denovo_dinuc_context_count_all_remake.pickle')

In [ ]:
denovo_dinuc_context_count_all = pd.read_pickle('denovo/denovo_dinuc_context_count_all_remake.pickle')

#### Count all 6-mer contexts in the sequenceable genome

In [ ]:
six_counts_mutregions = dict()
for chrom, pos in count_by_100kb.index:
    seq = hg38_genome[chrom][int(pos):int(pos+1e5)]
    six_counts_mutregions[(chrom, pos)] = pd.Series(re.findall('......', seq) + re.findall('......', seq[1:]) + re.findall('......', seq[2:]) + re.findall('......', seq[3:]) + re.findall('......', seq[4:]) + re.findall('......', seq[5:]), dtype = 'object').value_counts()
    print ('\r' + 'finished chr'+str(chrom) + ' '+str(pos), end = ' ')

In [ ]:
six_counts_mutregions_sum = pd.Series(dtype = 'int64')
counter = 0
for part in list(six_counts_mutregions.keys()):
    six_counts_mutregions_sum = six_counts_mutregions_sum.add(six_counts_mutregions[part], fill_value = 0)
    counter +=1; print('\r' + str(counter), end = ' ')
six_counts_mutregions_sum.to_pickle('denovo/sixlet_totals_hg38_mutregions_remake.pickle')

In [ ]:
six_counts_mutregions_sum = pd.read_pickle('denovo/sixlet_totals_hg38_mutregions_remake.pickle')

In [ ]:
six_counts_mutregions_sum_diindex = six_counts_mutregions_sum.copy()
six_counts_mutregions_sum_diindex.index = pd.MultiIndex.from_tuples([re.findall('..', seq) for seq in six_counts_mutregions_sum.index])
six_counts_mutregions_sum_diindex = six_counts_mutregions_sum_diindex.reset_index()

In [ ]:
six_counts_dinuc_context_count_all = pd.DataFrame(0, index = denovo_dinuc_context_count_all.index, columns = denovo_dinuc_context_count_all.columns)
for repeat in six_counts_dinuc_context_count_all.index:
    for motif in repeat_frames_RC(repeat):
        six_counts_dinuc_context_count_all.loc[repeat, 'Afission'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] == motif) & (six_counts_mutregions_sum_diindex['level_1'] == motif) & (six_counts_mutregions_sum_diindex['level_2'] == motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'Acontraction'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] == motif) & (six_counts_mutregions_sum_diindex['level_1'] == motif) & (six_counts_mutregions_sum_diindex['level_2'] != motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'Acontraction'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] != motif) & (six_counts_mutregions_sum_diindex['level_1'] == motif) & (six_counts_mutregions_sum_diindex['level_2'] == motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'A10'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] != motif) & (six_counts_mutregions_sum_diindex['level_1'] == motif) & (six_counts_mutregions_sum_diindex['level_2'] != motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'Afusion'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] == motif) & (six_counts_mutregions_sum_diindex['level_1'] != motif) & (six_counts_mutregions_sum_diindex['level_2'] == motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'Aexpansion'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] == motif) & (six_counts_mutregions_sum_diindex['level_1'] != motif) & (six_counts_mutregions_sum_diindex['level_2'] != motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'Aexpansion'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] != motif) & (six_counts_mutregions_sum_diindex['level_1'] != motif) & (six_counts_mutregions_sum_diindex['level_2'] == motif)][0].sum()
        six_counts_dinuc_context_count_all.loc[repeat, 'A01'] += six_counts_mutregions_sum_diindex.loc[(six_counts_mutregions_sum_diindex['level_0'] != motif) & (six_counts_mutregions_sum_diindex['level_1'] != motif) & (six_counts_mutregions_sum_diindex['level_2'] != motif)][0].sum()

#### Calculate rates

In [ ]:
denovo_substitution_context_rate_dinuc = denovo_dinuc_context_count_all /2 / six_counts_dinuc_context_count_all / denovo_n_genomes_snv
denovo_substitution_context_rate_dinuc.loc[['AC', 'AG', 'AT', 'CG']].to_pickle('denovo/denovo_mut_freq_dinuc_remake.pickle')

In [ ]:
denovo_substitution_context_rate = pd.concat([denovo_substitution_context_rate, denovo_substitution_context_rate_dinuc])

In [ ]:
denovo_substitution_context_rate.to_pickle('denovo/denovo_mut_freq_triplets_remake.pickle')

In [ ]:
denovo_mut_freq_dinuc_poisson = dict()
for i in range(200):
    denovo_mut_freq_dinuc_poisson[i] = (denovo_dinuc_context_count_all.dropna(axis=0) /2).apply(np.random.poisson) / six_counts_dinuc_context_count_all / denovo_n_genomes_snv
denovo_mut_freq_dinuc_poisson = pd.concat(denovo_mut_freq_dinuc_poisson).dropna(axis=0)
denovo_mut_freq_dinuc_poisson.to_pickle('denovo/denovo_mut_freq_dinuc_poisson_remake.pickle')

## put mono-, di- and trinucleotides into one file

In [ ]:
denovo_mut_freq_triplets = pd.read_pickle('denovo/denovo_mut_freq_triplets_remake.pickle')
denovo_mut_freq_dinuc = pd.read_pickle('denovo/denovo_mut_freq_dinuc_remake.pickle')
denovo_mut_freq_AB = pd.DataFrame(pd.read_pickle('denovo/denovo_mut_freq_AB_remake.pickle'), columns = ['A']).T
denovo_mut_freq_CD = pd.DataFrame(pd.read_pickle('denovo/denovo_mut_freq_CD_remake.pickle'), columns = ['C']).T
denovo_mut_freq_all = pd.concat([denovo_mut_freq_triplets, denovo_mut_freq_dinuc, denovo_mut_freq_AB, denovo_mut_freq_CD], axis=0).sort_index()

denovo_mut_freq_all.to_pickle('denovo/denovo_mut_freq_all.pickle')

In [ ]:
# poisson
denovo_mut_freq_triplets_poisson = pd.read_pickle('denovo/denovo_mut_freq_triplets_poisson_remake.pickle')
denovo_mut_freq_AB_poisson = pd.read_pickle('denovo/denovo_mut_freq_AB_poisson_remake.pickle').T
denovo_mut_freq_AB_poisson.index = pd.MultiIndex.from_tuples([(i, 'A') for i in denovo_mut_freq_AB_poisson.index])
denovo_mut_freq_CD_poisson = pd.read_pickle('denovo/denovo_mut_freq_CD_poisson_remake.pickle').T
denovo_mut_freq_CD_poisson.index = pd.MultiIndex.from_tuples([(i, 'C') for i in denovo_mut_freq_CD_poisson.index])
denovo_mut_freq_all_poisson = pd.concat([denovo_mut_freq_triplets_poisson, denovo_mut_freq_dinuc_poisson, denovo_mut_freq_AB_poisson, denovo_mut_freq_CD_poisson], axis=0).sort_index()

denovo_mut_freq_all_poisson.to_pickle('denovo/denovo_mut_freq_all_poisson.pickle')

In [ ]:
denovo_mut_freq_all = pd.read_pickle('denovo/denovo_mut_freq_all.pickle')
denovo_mut_freq_all_poisson = pd.read_pickle('denovo/denovo_mut_freq_all_poisson.pickle')

#### mu and nu rates
- overall A>B and B>A rates

In [ ]:
mu_all = dict(); nu_all = dict()
mu_all[1] = dict(); nu_all[1] = dict()
mu_all[1]['A'] = rates_mu_nu['B>A']
mu_all[1]['C'] = rates_mu_nu_CD['D>C']
mu_all[2] = denovo_dinuc_context_count_all[['Afusion', 'Aexpansion', 'A01']].sum(axis=1) / six_counts_dinuc_context_count_all[['Afusion', 'Aexpansion', 'A01']].sum(axis=1) / denovo_n_genomes_snv
mu_all[3] = denovo_triplet_context_count_all[['Afusion', 'Aexpansion', 'A01']].sum(axis=1) / nine_counts_triplet_context_count_all[['Afusion', 'Aexpansion', 'A01']].sum(axis=1) / denovo_n_genomes_snv
nu_all[1]['A'] = rates_mu_nu['A>B']
nu_all[1]['C'] = rates_mu_nu_CD['C>D']
nu_all[2] = denovo_dinuc_context_count_all[['Afission', 'Acontraction', 'A10']].sum(axis=1) / six_counts_dinuc_context_count_all[['Afission', 'Acontraction', 'A10']].sum(axis=1) / denovo_n_genomes_snv
nu_all[3] = denovo_triplet_context_count_all[['Afission', 'Acontraction', 'A10']].sum(axis=1) / nine_counts_triplet_context_count_all[['Afission', 'Acontraction', 'A10']].sum(axis=1) / denovo_n_genomes_snv

mu_nu_all = dict()
mu_nu_all['mu'] = pd.DataFrame(mu_all)
mu_nu_all['nu'] = pd.DataFrame(nu_all)
mu_nu_all = pd.concat(mu_nu_all, axis=1)

In [ ]:
mu_nu_all.to_pickle('denovo/mu_nu_1-3.pickle')

In [ ]:
mu_nu_all = pd.read_pickle('denovo/mu_nu_1-3.pickle')

# deCODE instability rate measurement
- Download supplementary data from Kristmundsdottir et al 2023 (see manuscript for DOI), extract in folder named 'decode'
- https://github.com/DecodeGenetics/mDNM_analysisAndData

### Process original data file
- used to calculate denominator for rates

In [ ]:
decode = pd.read_csv('decode/mDNM_analysisAndData-main/mutRateDataAll.gz', compression = 'gzip', sep = '\t', skiprows = 0)
decode = decode.reset_index()
decode.columns = ['chrom', 'start', 'end', 'motif', 'missing', 'DNMs', 'Correct', 'geneticDiversity', 'motifLength', 'refLen', 'purity', 'GCcontentInMotif']
decode['chrom'] = [chrom[3:] if chrom[3:] in ['X', 'Y'] else int(chrom[3:]) for chrom in decode['chrom']]
decode['motif_std'] = [repeat_frames_RC(rep)[0] for rep in decode['motif']]
decode['seq'] = [get_sequence(chrom, start, end, hg38_genome) for chrom, start, end in zip(decode['chrom'], decode['start'], decode['end'])]

In [ ]:
# Find longest pure repeat in sequence
decode['seq+-10'] = [get_sequence(chrom, start-10, end+10, hg38_genome) for chrom, start, end in zip(decode['chrom'], decode['start'], decode['end'])]
decode['max_pure_length'] = [max([len(rep) for rep in re.findall('(?:' + motif + ')+', seq)]) for motif, seq in zip(decode['motif'], decode['seq+-10'])]
decode['max_pure_units'] = decode['max_pure_length'] / decode['motifLength']
decode['ref_units'] = decode['refLen'] / decode['motifLength']
decode['ref_units_int'] = decode['ref_units'] //1
decode.to_pickle('decode/mutRateDataAll.pickle')

In [ ]:
decode = pd.read_pickle('decode/mutRateDataAll.pickle')

In [ ]:
# loci where popSTR-listed reference repeat length is a perfect repeat of the appropriate motif
decode.loc[decode['ref_units_int'] == decode['max_pure_units']]

In [ ]:
len(decode), len(decode.loc[decode['ref_units_int'] == decode['max_pure_units']])

In [ ]:
# 'Correct' column specifies number of families where the locus was sequenced with sufficient quality
# denominator by motif and reference length for rate calculations
# Because parental genotypes are not available for all loci (just for mutated loci), the denominator is approximated using this distribution of reference length counts.
motif_counts_pure_units = decode.loc[decode['ref_units_int'] == decode['max_pure_units']].groupby(['ref_units_int', 'motif_std'])['Correct'].sum().unstack()

### Process extended data file
- used to calculate numerator for rates

In [ ]:
dec_ext = pd.read_csv('decode/mDNM_analysisAndData-main/bpInvolved_extended', sep = '\t')

dec_ext[['Proband_1', 'Proband_2']] = (dec_ext['ProbandGt'].str.split('/', expand = True)).astype(float).round(1)
dec_ext[['Father_1', 'Father_2']] = (dec_ext['FatherGt'].str.split('/', expand = True)).astype(float).round(1)
dec_ext[['Mother_1', 'Mother_2']] = (dec_ext['MotherGt'].str.split('/', expand = True)).astype(float).round(1)
dec_ext['Motif_len'] = dec_ext['Motif'].str.len()
dec_ext['Phase'] = dec_ext['Phase'].replace('X', 0).astype(int)
dec_ext['units_Involved'] = (dec_ext['BpInvolved'].replace('X', np.nan).astype(float) / dec_ext['Motif_len']).round(1)
dec_ext['exp_con'] = ['exp' if change > 0 else 'con' if change < 0 else np.nan for change in dec_ext['units_Involved']]

dec_ext['motif_std'] = [repeat_frames_RC(rep)[0] for rep in dec_ext['Motif']]
dec_ext['Chrom'] = [chrom[3:] if chrom[3:] in ['X', 'Y'] else int(chrom[3:]) for chrom in dec_ext['Chrom']]

In [ ]:
# get sequence upstream and downstream of mutation position based on given hg38 RefLen
dec_ext['seq'] = [get_sequence(chrom, pos-reflen-10, pos+reflen+10, hg38_genome) for chrom, pos, reflen in zip(dec_ext['Chrom'], dec_ext['Pos'], dec_ext['RefLen'])]
# Find longest pure repeat in sequence
dec_ext['RefLen_pure'] = [max([len(rep) for rep in re.findall('(?:' + motif + ')+', seq)]) for motif, seq in zip(dec_ext['Motif'], dec_ext['seq'])]
dec_ext['RefLen_units'] = dec_ext['RefLen'] / dec_ext['Motif_len']
dec_ext['RefLen_units_pure'] = dec_ext['RefLen_pure'] / dec_ext['Motif_len']

#### Find closest parental allele
in the absence of phasing, assume mutation originated from parental allele with nearest length

In [ ]:
def check_p(F1, UI, P1):
    return np.abs(F1+UI-P1) <= 0.11
dec_ext['parent_allele'] = ['F1' if check_p(F1,UI,P1) & (phase == 1) else 'F2' if check_p(F2,UI,P1) & (phase == 1) else 'F1' if check_p(F1,UI,P2) & (phase == 1) else 'F2' if check_p(F2,UI,P2) & (phase == 1) else 'M1' if check_p(M1,UI,P1) & (phase == 2) else 'M2' if check_p(M2,UI,P1) & (phase == 2) else 'M1' if check_p(M1,UI,P2) & (phase == 2) else 'M2' if check_p(M2,UI,P2) & (phase == 2) else 'unknown' for F1, F2, M1, M2, P1, P2, UI, phase in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['Proband_1'], dec_ext['Proband_2'], dec_ext['units_Involved'], dec_ext['Phase'])]

In [ ]:
dec_ext['P1_in_M'] = [(P1 in [M1, M2]) for M1, M2, P1, P2 in zip(dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['Proband_1'], dec_ext['Proband_2'])]
dec_ext['P2_in_M'] = [(P2 in [M1, M2]) for M1, M2, P1, P2 in zip(dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['Proband_1'], dec_ext['Proband_2'])]
dec_ext['P1_in_F'] = [(P1 in [F1, F2]) for F1, F2, P1, P2 in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Proband_1'], dec_ext['Proband_2'])]
dec_ext['P2_in_F'] = [(P2 in [F1, F2]) for F1, F2, P1, P2 in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Proband_1'], dec_ext['Proband_2'])]

dec_ext['P1F_P2F_P1M_P2M'] = dec_ext[['P1_in_F','P2_in_F', 'P1_in_M', 'P2_in_M']].astype(int).values.tolist()
dec_ext['P1F_P2F_P1M_P2M'] = [''.join(str(val)) for val in dec_ext['P1F_P2F_P1M_P2M']]

# find probable parental allele contributing to instability
# assumes the instable allele comes from the closest length parental allele, after identifying the stable allele by a match between any proband and parental allele
dec_ext['probable_parent_allele'] = ['F1' if (min([abs(P1-F1), abs(P2-F1)]) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else 
'F2' if (min([abs(P1-F2), abs(P2-F2)]) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else
'M1' if (min([abs(P1-M1), abs(P2-M1)]) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'M2' if (min([abs(P1-M2), abs(P2-M2)]) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'F1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'F2' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'F1' if (abs(P2-F1) == min([abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 0]') else 
'F2' if (abs(P2-F2) == min([abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 0]') else 
'M1' if (abs(P1-M1) == min([abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 0]') else 
'M2' if (abs(P1-M2) == min([abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 0]') else 
'M1' if (abs(P2-M1) == min([abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 0, 0]') else 
'M2' if (abs(P2-M2) == min([abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 0, 0]') else 
'F1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'F2' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'F1' if (abs(P2-F1) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'F2' if (abs(P2-F2) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'M1' if (abs(P2-M1) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'M2' if (abs(P2-M2) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'F1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'F2' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'M1' if (abs(P1-M1) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'M2' if (abs(P1-M2) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 'unknown'
 for F1,  F2, M1,  M2, P1,  P2, FFMM in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['Proband_1'], dec_ext['Proband_2'], dec_ext['P1F_P2F_P1M_P2M'])]

# find probable proband allele contributing to instability
# assumes the instable allele comes from the closest length parental allele, after identifying the stable allele by a match between any proband and parental allele
dec_ext['probable_proband_allele'] = [
'P1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else 
'P2' if (abs(P2-F1) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else 
'P1' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else
'P2' if (abs(P2-F2) == min([abs(P1-F1), abs(P1-F2), abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 1]') else
'P1' if (abs(P1-M1) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'P2' if (abs(P2-M1) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'P1' if (abs(P1-M2) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'P2' if (abs(P2-M2) == min([abs(P1-M1), abs(P1-M2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 1, 0, 0]') else 
'P1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'P1' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'P2' if (abs(P2-F1) == min([abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 0]') else 
'P2' if (abs(P2-F2) == min([abs(P2-F1), abs(P2-F2)])) & (FFMM == '[0, 0, 1, 0]') else 
'P1' if (abs(P1-M1) == min([abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 0]') else 
'P1' if (abs(P1-M2) == min([abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 0]') else 
'P2' if (abs(P2-M1) == min([abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 0, 0]') else 
'P2' if (abs(P2-M2) == min([abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 0, 0]') else 
'P1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'P1' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2)])) & (FFMM == '[0, 0, 0, 1]') else 
'P2' if (abs(P2-F1) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'P2' if (abs(P2-F2) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'P2' if (abs(P2-M1) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'P2' if (abs(P2-M2) == min([abs(P2-F1), abs(P2-F2), abs(P2-M1), abs(P2-M2)])) & (FFMM == '[1, 0, 1, 0]') else 
'P1' if (abs(P1-F1) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'P1' if (abs(P1-F2) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'P1' if (abs(P1-M1) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 
'P1' if (abs(P1-M2) == min([abs(P1-F1), abs(P1-F2), abs(P1-M1), abs(P1-M2)])) & (FFMM == '[0, 1, 0, 1]') else 'unknown'
 for F1,  F2, M1,  M2, P1,  P2, FFMM in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['Proband_1'], dec_ext['Proband_2'], dec_ext['P1F_P2F_P1M_P2M'])]

In [ ]:
# use known allele when available, and likely allele if not
dec_ext['known_or_likely_parent_allele'] = [parent if parent != 'unknown' else probable for parent, probable in zip(dec_ext['parent_allele'], dec_ext['probable_parent_allele'])]
dec_ext['known_or_likely_parent_length'] = [F1 if likely == 'F1' else F2 if likely == 'F2' else M1 if likely == 'M1' else M2 if likely == 'M2' else np.nan for F1, F2, M1, M2, likely in zip(dec_ext['Father_1'], dec_ext['Father_2'], dec_ext['Mother_1'], dec_ext['Mother_2'], dec_ext['known_or_likely_parent_allele'])]
# round down unit length of partial repeats
dec_ext['known_or_likely_parent_length_int'] = dec_ext['known_or_likely_parent_length'] //1

dec_ext['likely_BpInvolved'] = [P1 - parent if likely == 'P1' else P2 - parent if likely == 'P2' else np.nan for P1, P2, likely, parent, in zip(dec_ext['Proband_1'], dec_ext['Proband_2'], dec_ext['probable_proband_allele'], dec_ext['known_or_likely_parent_length'])]
dec_ext['known_or_likely_BpInvolved'] = [float(bp) if bp != 'X' else probable for bp, probable in zip(dec_ext['BpInvolved'], dec_ext['likely_BpInvolved'])]
dec_ext['known_or_likely_units_Involved'] = dec_ext['known_or_likely_BpInvolved'] / dec_ext['Motif_len']
dec_ext['known_or_likely_exp_con'] = ['exp' if change > 0 else 'con' if change < 0 else 'unknown' for change in dec_ext['known_or_likely_units_Involved']]

In [ ]:
dec_ext.to_pickle('decode/DECODE_extended_data.pickle')

In [ ]:
dec_ext = pd.read_pickle('decode/DECODE_extended_data.pickle')

In [ ]:
dec_ext['ref_units'] = dec_ext['RefLen'] / dec_ext['Motif_len']
dec_ext['ref_units_int'] = dec_ext['ref_units'] //1
dec_ext_pure = dec_ext.loc[(dec_ext['ref_units_int'] == dec_ext['RefLen_units_pure'])].copy()

dec_ext_pure['Mother_1_int'] = dec_ext_pure['Mother_1'] // 1
dec_ext_pure['Mother_2_int'] = dec_ext_pure['Mother_2'] // 1
dec_ext_pure['Father_1_int'] = dec_ext_pure['Father_1'] // 1
dec_ext_pure['Father_2_int'] = dec_ext_pure['Father_2'] // 1

In [ ]:
len(dec_ext), len(dec_ext_pure)

In [ ]:
# choose only single unit indels
dec_ext_pure = dec_ext_pure.loc[dec_ext_pure['known_or_likely_units_Involved'].abs() == 1]
len(dec_ext_pure)

### Calculate rates

In [ ]:
# DECODE data
decode_exp_rate = dict(); decode_con_rate = dict()
decode_exp_counts = dict(); decode_con_counts = dict()
decode_exp_rate_poisson = dict(); decode_con_rate_poisson = dict()
for motif in reps_1_4:
    # using closest parental allele 
    decode_exp_counts[motif] = dec_ext_pure.loc[(dec_ext_pure['motif_std'] == motif) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['known_or_likely_parent_length_int'].value_counts()
    decode_con_counts[motif] = dec_ext_pure.loc[(dec_ext_pure['motif_std'] == motif) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['known_or_likely_parent_length_int'].value_counts()
    # assuming reference length is parental length, but size and direction of change (units_involved) is the same
#    decode_exp_counts[motif] = dec_ext_pure.loc[(dec_ext_pure['motif_std'] == motif) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['ref_units_int'].value_counts()
#    decode_con_counts[motif] = dec_ext_pure.loc[(dec_ext_pure['motif_std'] == motif) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['ref_units_int'].value_counts()

    loci_count = motif_counts_pure_units[motif] *2 # number of trios where all three members pass quality filters, multiplied by 2 parental alleles
    # using DECODE population information for denominator, with adjustment for average number of motifs passing filters / total number of motifs
    decode_exp_rate[motif] = decode_exp_counts[motif].div(loci_count).dropna() 
    decode_con_rate[motif] = decode_con_counts[motif].div(loci_count).dropna()
    if len(decode_exp_rate[motif]) > 0:
        decode_exp_rate_poisson_current = dict()
        for i in range(200):
            decode_exp_rate_poisson_current[i] = decode_exp_counts[motif].apply(lambda x: np.random.poisson(x)).div(loci_count)
        decode_exp_rate_poisson[motif] = pd.concat(decode_exp_rate_poisson_current)#.div(decode_exp_rate_poisson_current.index * len(motif))
    if len(decode_con_rate[motif]) > 0:
        decode_con_rate_poisson_current = dict()
        for i in range(200):
            decode_con_rate_poisson_current[i] = decode_con_counts[motif].apply(lambda x: np.random.poisson(x)).div(loci_count)
        decode_con_rate_poisson[motif] = pd.concat(decode_con_rate_poisson_current)
decode_exp_counts = pd.concat(decode_exp_counts, axis=1).sort_index()
decode_con_counts = pd.concat(decode_con_counts, axis=1).sort_index()
decode_exp_rate = pd.concat(decode_exp_rate, axis=1).sort_index()
decode_con_rate = pd.concat(decode_con_rate, axis=1).sort_index()
decode_exp_rate_poisson = pd.concat(decode_exp_rate_poisson, axis=1).sort_index()
decode_con_rate_poisson = pd.concat(decode_con_rate_poisson, axis=1).sort_index()

In [ ]:
# combine units
for unit_len in [1,2,3,4]:
    current_reps = [rep for rep in decode_exp_rate.columns if len(rep) == unit_len]
    decode_exp_counts['unit_'+str(unit_len)] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(current_reps)) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['known_or_likely_parent_length_int'].value_counts()
    decode_con_counts['unit_'+str(unit_len)] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(current_reps)) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['known_or_likely_parent_length_int'].value_counts()
    loci_count_units = motif_counts_pure_units[current_reps].sum(axis=1) *2
    # using DECODE parental information for denominator, with adjustment for average number of motifs passing filters / total number of motifs
    decode_exp_rate['unit_'+str(unit_len)] = decode_exp_counts['unit_'+str(unit_len)].div(loci_count_units).dropna() 
    decode_con_rate['unit_'+str(unit_len)] = decode_con_counts['unit_'+str(unit_len)].div(loci_count_units).dropna()
    
    decode_exp_rate_units = dict(); decode_con_rate_units = dict()
    for i in range(200):
        decode_exp_rate_units[i] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(current_reps)) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['known_or_likely_parent_length_int'].value_counts().apply(np.random.poisson).div(loci_count_units).dropna()
        decode_con_rate_units[i] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(current_reps)) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['known_or_likely_parent_length_int'].value_counts().apply(np.random.poisson).div(loci_count_units).dropna()
    decode_exp_rate_poisson['unit_'+str(unit_len)] = pd.concat(decode_exp_rate_units, axis=0).replace(0, np.nan)
    decode_con_rate_poisson['unit_'+str(unit_len)] = pd.concat(decode_con_rate_units, axis=0).replace(0, np.nan)

In [ ]:
# combine non-CpG trinucleotide repeats
decode_exp_count_tri_noCpG = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(reps_tri_noCpG)) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['known_or_likely_parent_length_int'].value_counts()
decode_con_count_tri_noCpG = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(reps_tri_noCpG)) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['known_or_likely_parent_length_int'].value_counts()
loci_count_tri_noCpG = motif_counts_pure_units[reps_tri_noCpG].sum(axis=1) *2
# using DECODE parental information for denominator, with adjustment for average number of motifs passing filters / total number of motifs
decode_exp_rate['tri_noCpG'] = decode_exp_count_tri_noCpG.div(loci_count_tri_noCpG).dropna() 
decode_con_rate['tri_noCpG'] = decode_con_count_tri_noCpG.div(loci_count_tri_noCpG).dropna()

decode_exp_rate_tri_noCpG = dict(); decode_con_rate_tri_noCpG = dict()
for i in range(200):
    decode_exp_rate_tri_noCpG[i] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(reps_tri_noCpG)) & (dec_ext_pure['known_or_likely_exp_con'] == 'exp')]['known_or_likely_parent_length_int'].value_counts().apply(np.random.poisson).div(loci_count_tri_noCpG).dropna()
    decode_con_rate_tri_noCpG[i] = dec_ext_pure.loc[(dec_ext_pure['motif_std'].isin(reps_tri_noCpG)) & (dec_ext_pure['known_or_likely_exp_con'] == 'con')]['known_or_likely_parent_length_int'].value_counts().apply(np.random.poisson).div(loci_count_tri_noCpG).dropna()
decode_exp_rate_poisson['tri_noCpG'] = pd.concat(decode_exp_rate_tri_noCpG, axis=0).replace(0, np.nan)
decode_con_rate_poisson['tri_noCpG'] = pd.concat(decode_con_rate_tri_noCpG, axis=0).replace(0, np.nan)

In [ ]:
# not adjusting for motif length here... calculate per unit instead of per nt
decode_exp_rate = decode_exp_rate.div(decode_exp_rate.index, axis=0) 
decode_con_rate = decode_con_rate.div(decode_con_rate.index, axis=0)
decode_exp_rate_poisson = decode_exp_rate_poisson.div(decode_exp_rate_poisson.index.get_level_values(1), axis=0)
decode_con_rate_poisson = decode_con_rate_poisson.div(decode_con_rate_poisson.index.get_level_values(1), axis=0)

In [ ]:
decode_exp_counts.to_pickle('decode/decode_expansion_counts.pickle')
decode_con_counts.to_pickle('decode/decode_contraction_counts.pickle')

In [ ]:
decode_exp_rate.to_pickle('decode/decode_expansion_rates.pickle')
decode_con_rate.to_pickle('decode/decode_contraction_rates.pickle')

decode_exp_rate_poisson.to_pickle('decode/decode_expansion_rates_poisson.pickle')
decode_con_rate_poisson.to_pickle('decode/decode_contraction_rates_poisson.pickle')

In [ ]:
decode_exp_counts = pd.read_pickle('decode/decode_expansion_counts.pickle')
decode_con_counts = pd.read_pickle('decode/decode_contraction_counts.pickle')

decode_exp_rate = pd.read_pickle('decode/decode_expansion_rates.pickle')
decode_con_rate = pd.read_pickle('decode/decode_contraction_rates.pickle')

decode_exp_rate_poisson = pd.read_pickle('decode/decode_expansion_rates_poisson.pickle')
decode_con_rate_poisson = pd.read_pickle('decode/decode_contraction_rates_poisson.pickle')

decode_con_rate_poisson = decode_con_rate_poisson.reindex(decode_exp_rate_poisson.columns, axis=1)

# Figures

#### Units added per event

In [ ]:
dec_ext['BpInvolved_withsign'] = dec_ext['BpInvolved'].replace('X', np.nan).astype(float)
dec_ext['BpInvolved_abs'] = np.abs(dec_ext['BpInvolved_withsign'])

In [ ]:
decode_unit_len_per_event = dec_ext.dropna(subset = ['BpInvolved_withsign']).groupby(['Motif_len', 'RefLen_pure', 'BpInvolved_withsign'])['Chrom'].count().unstack().div(dec_ext.groupby(['Motif_len', 'RefLen_pure', 'BpInvolved_abs'])['Chrom'].count().unstack().sum(axis=1), axis=0)

In [ ]:
# remove any decode measurement below 10 nt
decode_unit_len_per_event.loc[1,[5,6,7,8,9],:] = np.nan
decode_unit_len_per_event.loc[2,[6,8],:] = np.nan
decode_unit_len_per_event.loc[3,[6,9],:] = np.nan
decode_unit_len_per_event.loc[4,[4,8],:] = np.nan

In [ ]:
# Fig. S4 (bottom)
fig = make_subplots(rows = 4, cols = 1, shared_xaxes = True, vertical_spacing = 0.015, row_titles = ['unit=1', 'unit=2', 'unit=3', 'unit=4'], x_title = 'repeat tract length (units)', y_title = 'fraction of events per length bin')
for unit_len in range(1,5):
    current_units = decode_unit_len_per_event.loc[unit_len]
    for inslen in [1,2,3,4]:
        fig.add_trace(go.Bar(x = current_units.index / unit_len, y = current_units[inslen], name = str(np.abs(inslen)), legendgroup = str(np.abs(inslen)), showlegend = True if unit_len == 1 else False), row = unit_len, col = 1)
    fig.add_trace(go.Bar(x = current_units.index  / unit_len, y = current_units[[col for col in current_units.columns if col > 4]].sum(axis=1), name = '>4', legendgroup = 'other', showlegend = True if unit_len == 1 else False), row = unit_len, col = 1)
    for inslen in [-1,-2,-3,-4]:
        fig.add_trace(go.Bar(x = current_units.index / unit_len, y = -current_units[inslen], name = str(np.abs(inslen)), legendgroup = str(np.abs(inslen)), showlegend = False), row = unit_len, col = 1)
    fig.add_trace(go.Bar(x = current_units.index  / unit_len, y = -current_units[[col for col in current_units.columns if col < -4]].sum(axis=1), name = '>4', legendgroup = 'other', showlegend = False), row = unit_len, col = 1)
    fig.update_xaxes(range = [2.5,33.5], dtick = 1, row = unit_len, col = 1)
fig.update_yaxes(range = [-1,1], tickvals = [-0.5, 0, 0.5])
fig.update_layout(font=dict(family = 'Arial', size = 14), height = 400, width = 1000, title = 'popSTR', colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:5], legend_title_text='nt added', margin={'t':30,'l':60,'b':60,'r':10})
fig.update_layout(barmode = 'relative')
fig.show()

In [ ]:
fig.write_image('plots/figS4b.pdf')

In [ ]:
# Fig. 2a
fig = go.Figure()
for unit in [1,2,3,4]:
    current_focus = (unit_fraction_of_events_per_length[unit].dropna(how = 'all', axis=1)[unit] + unit_fraction_of_events_per_length[unit].dropna(how = 'all', axis=1)[-unit]) / unit_fraction_of_events_per_length[unit].dropna(how = 'all', axis=1).sum(axis=1)
    if unit ==1:
        current_focus = current_focus.reindex(range(1,11))
    if unit ==2:
        current_focus = current_focus.reindex(range(1,9))
    if unit ==3:
        current_focus = current_focus.reindex(range(1,8))
    if unit ==4:
        current_focus = current_focus.reindex(range(1,6))
    fig.add_trace(go.Scatter(x = current_focus.index, y = current_focus, mode = 'markers', marker = dict(symbol = 'square-open', line_width = 3, size = 7), legendgroup = unit, name = 'unit=' +str(unit) + ' (pooled trios)'))

for unit in [1,2,3,4]:    
    current_focus = (decode_unit_len_per_event.loc[unit].dropna(how = 'all', axis=1)[unit] + decode_unit_len_per_event.loc[unit].dropna(how = 'all', axis=1)[-unit]) / decode_unit_len_per_event.loc[unit].dropna(how = 'all', axis=1).sum(axis=1)
    current_focus.index /= unit
    if unit ==1:
        current_focus = current_focus.reindex(range(10,26))
    if unit ==2:
        current_focus = current_focus.reindex(range(5,31))
    if unit ==3:
        current_focus = current_focus.reindex(range(4,31))
    if unit ==4:
        current_focus = current_focus.reindex(range(4,31))
    fig.add_trace(go.Scatter(x = current_focus.index, y = current_focus, mode = 'markers', marker = dict(symbol = 'circle-open', line_width = 3, size = 7), legendgroup = unit, name = 'unit=' +str(unit) + ' (popSTR)'))

fig.update_xaxes(title = 'repeat tract length (units)')
fig.update_yaxes(title = 'fraction +/-1 unit indels')
fig.update_layout(font=dict(family = 'Arial', size = 14), height = 300, width = 550, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:4], margin={'t':30,'l':60,'b':40,'r':10})
fig.show()

In [ ]:
fig.write_image('plots/fig2b_unitlengthchange.pdf')

#### Rate curve plots

In [ ]:
def add_rate_trace(rates, name, symbol, colornum, motif='A', rowcol=None, markersize = 7, per = 'unit', xmult = 1, plot_min = 1e-12, plot_range = range(1,32)):
    rates = rates[motif].unstack().replace(np.inf, np.nan).dropna(how='all', axis=1).loc[1:]
    med = rates.median(axis=0).replace(0, np.nan).replace(np.inf, np.nan).dropna().reindex(plot_range)
    low = med.sub(rates.quantile(0.025).replace(0, np.nan).replace(np.inf, np.nan)).reindex(plot_range)
    high = (rates.quantile(0.975).replace(0, np.nan).replace(np.inf, np.nan)).sub(med).reindex(plot_range)
    
    if per == 'repeat':
        med *= med.index; low *= low.index; high *= high.index
    if per == 'nt':
        med *= len(motif); low *= len(motif); high *= len(motif)

    low = low.replace(0, plot_min)

    if rowcol is None:
        fig_rates.add_trace(go.Scatter(x = med.index * xmult, y = med, error_y = dict(type = 'data', arrayminus = low, array = high, color = 'rgba(0,0,0,0.2)', thickness = (1.8/7) * markersize, width = markersize), name = name, mode = 'markers', opacity = 0.95, marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[colornum], symbol= symbol, size = markersize, line_width = (3/7) * markersize)))
    else:
        fig_rates.add_trace(go.Scatter(x = med.index * xmult, y = med, error_y = dict(type = 'data', arrayminus = low, array = high, color = 'rgba(0,0,0,0.2)', thickness = (1.8/7) * markersize, width = markersize), name = name, mode = 'markers', opacity = 0.95, marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[colornum], symbol= symbol, size = markersize, line_width = (3/7) * markersize), showlegend = True if (rowcol[0]==1) & (rowcol[1]==1) else False), row = rowcol[0], col = rowcol[1])

In [ ]:
# Fig. 2b
fig_rates = go.Figure()
add_rate_trace(denovo_exp_rate_poisson, 'Expansion (pooled trios)', 'square-open', colornum = 0, plot_range=range(1,9))
add_rate_trace(denovo_con_rate_poisson, 'Contraction (pooled trios)', 'square-open', colornum = 1, plot_range=range(1,9))
add_rate_trace(denovo_nonexp_rate_poisson, 'Non-motif insertion (pooled trios)', 'square-open', colornum = 2, plot_range=range(2,11))
add_rate_trace(decode_exp_rate_poisson, 'Expansion/insertion (popSTR)', 'circle-open', colornum = 0)
add_rate_trace(decode_con_rate_poisson, 'Contraction (popSTR)', 'circle-open', colornum = 1)

fig_rates.add_trace(go.Scatter(x = list(range(1,40)), y = [mu_nu_all['nu'][1]['A']]*40, name = 'substitutions A>B', line = dict(color = 'rgba(50,50,50,0.5)', dash = 'dash'), showlegend = False))
fig_rates.add_trace(go.Scatter(x = list(range(1,40)), y = [mu_nu_all['mu'][1]['A']]*40, name = 'substitutions B>A', line = dict(color = 'rgba(50,50,50,0.5)', dash = 'dot'), showlegend = False))

fig_rates.update_xaxes(type = 'log', title = 'repeat tract length (units)', range = [-0.03,1.6], tickvals = [1,5,10,20,30], gridcolor = 'rgba(0,0,0,0.2)')
fig_rates.update_yaxes(type = 'log', title = 'rate (per nt per generation)', range = [-11.6, -4.8], exponentformat =  'e', dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig_rates.update_layout(margin={'t':20,'l':80,'b':45,'r':10}, height = 360, width = 540)
fig_rates.update_layout(font=dict(family = 'Arial', size = 15), legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.023))

In [ ]:
fig_rates.write_image('plots/fig2b_rates.pdf')

In [ ]:
# Fig. S5a
motif_list = ['A', 'C', 'AC', 'AG', 'AT', 'CG', 'AAC', 'AAG', 'AAT', 'ACC', 'ACT', 'AGC', 'AGG', 'ATC', 'CCG', 'AAAC', 'AAAG', 'AAAT', 'AAGG', 'AATG', 'ACAG', 'ACAT', 'AGAT', 'ATCC']
fig_rates = make_subplots(rows = 6, cols = 4, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.025, vertical_spacing = 0.05, subplot_titles = motif_list, x_title = 'repeat tract length (units)', y_title = 'rate (per nt per generation)')
col_counter = 0; row_counter = 1
for motif in motif_list:
    col_counter +=1
    if col_counter ==5:
        row_counter +=1; col_counter -=4
    add_rate_trace(denovo_exp_rate_poisson, 'Expansion (pooled trios)', 'square-open', colornum = 0, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(decode_exp_rate_poisson, 'Expansion/insertion (popSTR)', 'circle-open', colornum = 0, motif = motif, plot_range = range(round((10/len(motif)) + 0.49),30), markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(denovo_con_rate_poisson, 'Contraction (pooled trios)', 'square-open', colornum = 1, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(decode_con_rate_poisson, 'Contraction (popSTR)', 'circle-open', colornum = 1, motif = motif, plot_range = range(round((10/len(motif)) + 0.49),30), markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(denovo_nonexp_rate_poisson, 'Non-motif insertion (pooled trios)', 'square-open', colornum = 2, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
fig_rates.update_xaxes(type = 'log', range = [-0.1,2], tickvals = [1,5,10,25])
fig_rates.update_yaxes(type = 'log', range = [-12.1, -2.4], exponentformat =  'e', dtick = 2)
fig_rates.update_layout(font=dict(family = 'Arial', size = 12), margin={'t':20,'l':80,'b':70,'r':10}, legend = dict(orientation = 'h', xanchor = 'center', x = 0.5, y= -0.08), height = 840, width = 640, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:3])

In [ ]:
fig_rates.write_image('plots/figS5a_rates.pdf')

In [ ]:
# Fig. S5b
motif_list = ['unit_1', 'unit_2', 'unit_3', 'unit_4']
fig_rates = make_subplots(rows = 4, cols = 1, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.025, vertical_spacing = 0.05, subplot_titles = ['unit length = 1', 'unit length = 2', 'unit length = 3', 'unit length = 4'], x_title = 'repeat tract length (units)', y_title = 'rate (per nt per generation)')
col_counter = 1; row_counter = 0
for motif in motif_list:
    row_counter +=1
    add_rate_trace(denovo_exp_rate_poisson, 'Expansion (pooled trios)', 'square-open', colornum = 0, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(decode_exp_rate_poisson, 'Expansion/insertion (popSTR)', 'circle-open', colornum = 0, motif = motif, plot_range = range(round((10/int(motif[-1])) + 0.49),30), markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(denovo_con_rate_poisson, 'Contraction (pooled trios)', 'square-open', colornum = 1, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(decode_con_rate_poisson, 'Contraction (popSTR)', 'circle-open', colornum = 1, motif = motif, plot_range = range(round((10/int(motif[-1])) + 0.49),30), markersize = 4, rowcol = (row_counter, col_counter))
    add_rate_trace(denovo_nonexp_rate_poisson, 'Non-motif insertion (pooled trios)', 'square-open', colornum = 2, motif = motif, markersize = 4, rowcol = (row_counter, col_counter))
fig_rates.update_xaxes(type = 'log', range = [-0.1,2], tickvals = [1,5,10,25])
fig_rates.update_yaxes(type = 'log', range = [-12.1, -2.4], exponentformat =  'e', dtick = 2)
fig_rates.update_layout(font=dict(family = 'Arial', size = 12), margin={'t':20,'l':80,'b':70,'r':10}, legend = dict(orientation = 'h', xanchor = 'center', x = 0.5, y= -0.08), height = 840, width = 540, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:3])

In [ ]:
fig_rates.write_image('plots/figS5b_rates.pdf')

#### save rate data for Supplementary File SF2

In [ ]:
SF2_data = dict()
SF2_data['denovo_substitutions'] = denovo_mut_freq_all.transpose()
SF2_data['denovo_expansions'] = denovo_exp_rate_poisson.unstack().median(axis=0).unstack().transpose().replace(0, np.nan).replace(np.inf, np.nan).dropna(how='all')
SF2_data['denovo_contractions'] = denovo_con_rate_poisson.unstack().median(axis=0).unstack().transpose().replace(0, np.nan).replace(np.inf, np.nan).dropna(how='all')
SF2_data['denovo_non_motif_insertions'] = denovo_nonexp_rate_poisson.unstack().median(axis=0).unstack().transpose().replace(0, np.nan).replace(np.inf, np.nan).dropna(how='all')
SF2_data['popSTR_insertions'] = decode_exp_rate_poisson.unstack().median(axis=0).unstack().transpose().replace(0, np.nan).replace(np.inf, np.nan).dropna(how='all')
SF2_data['popSTR_contractions'] = decode_con_rate_poisson.unstack().median(axis=0).unstack().transpose().replace(0, np.nan).replace(np.inf, np.nan).dropna(how='all')

In [ ]:
SF2_data['denovo_substitutions'].index = ['AAA>ABA', 'AAB>ABB', 'BAB>BBB', 'ABA>AAA', 'ABB>AAB', 'BBB>BAB']

In [ ]:
writer = pd.ExcelWriter('decode/SF2_rate_data.xlsx')
for sheet, frame in  SF2_data.items():
    frame.to_excel(writer, sheet_name = sheet)
writer.close()

# regression on popSTR data

## setup for all curve fits

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# note: combination of two lists
intercept_list = [denovo_con_rate['A'][8]] + [denovo_exp_rate['A'][8] * (denovo_exp_rate['A'][8]/ denovo_con_rate['A'][8])**x for x in range(7)]

In [ ]:
# use all popSTR points, inflate error on untrusted points
exp_std = decode_exp_rate_poisson['A'].unstack().std().dropna()
exp_std.loc[22:] *=100
exp_std.loc[11:12] *=100

con_std = decode_con_rate_poisson['A'].unstack().std().dropna()
con_std.loc[20:] *=100
con_std.loc[11:12] *=100

## Linear curve fit

In [ ]:
def func_lin(x, m, c):
    return m * x + c

In [ ]:
exp_fit, exp_pcov = curve_fit(func_lin, np.log10(decode_exp_rate['A'].dropna().index), np.log10(decode_exp_rate['A'].dropna()), sigma = np.log10(exp_std))
con_fit, con_pcov = curve_fit(func_lin, np.log10(decode_con_rate['A'].dropna().index), np.log10(decode_con_rate['A'].dropna()), sigma = np.log10(con_std))

In [ ]:
# Fig. S6a
fig_rates = go.Figure()
add_rate_trace(denovo_exp_rate_poisson, 'Exp. (pooled trios)', 'square-open', colornum = 0, plot_range=range(1,9))
add_rate_trace(denovo_con_rate_poisson, 'Con. (pooled trios)', 'square-open', colornum = 1, plot_range=range(1,9))
#add_rate_trace(denovo_nonexp_rate_poisson, 'Ins. (de novo)', 'square-open')
add_rate_trace(decode_exp_rate_poisson, 'Exp. (popSTR)', 'circle-open', colornum = 0)
add_rate_trace(decode_con_rate_poisson, 'Con. (popSTR)', 'circle-open', colornum = 1)
fig_rates.update_xaxes(type = 'log', title = 'repeat tract length (nt)', range = [-0.03,1.8], tickvals = [1,2,5,10,20,50], gridcolor = 'rgba(0,0,0,0.2)')
fig_rates.update_yaxes(type = 'log', title = 'rate (per nt per generation)', range = [-11.6, -4.5], exponentformat =  'e', dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig_rates.update_layout(margin={'t':20,'l':60,'b':35,'r':10}, height = 450, width = 600, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:2])
#fig_rates.update_layout(legend=dict(yanchor="top", y=0.97, xanchor="left", x=0.075))

mean_exp = pd.Series([10**(func_lin(np.log10(L), (exp_fit[0]), (exp_fit[1]))) for L in range(5,60)], index = list(range(5,60)))
upper_exp = pd.Series([10**func_lin(np.log10(L), exp_fit[0] + 2*np.sqrt(exp_pcov[0][0]), exp_fit[1] - 2*np.sqrt(exp_pcov[1][1])) for L in range(5,60)], index = list(range(5,60)))
lower_exp = pd.Series([10**func_lin(np.log10(L), exp_fit[0] - 2*np.sqrt(exp_pcov[0][0]), exp_fit[1] + 2*np.sqrt(exp_pcov[1][1])) for L in range(5,60)], index = list(range(5,60)))

fig_rates.add_trace(go.Scatter(x = upper_exp.index, y = upper_exp, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 0), name = 'CI', showlegend = False, legendgroup = 'exp_regress'))
fig_rates.add_trace(go.Scatter(x = mean_exp.index, y = mean_exp, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]), name = 'Exp. PL fit', legendgroup = 'exp_regress', fill = 'tonexty'))
fig_rates.add_trace(go.Scatter(x = lower_exp.index, y = lower_exp, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 0), name = 'CI', showlegend = False,  legendgroup = 'exp_regress', fill = 'tonexty'))

mean_con = pd.Series([10**func_lin(np.log10(L), con_fit[0], con_fit[1]) for L in range(5,60)], index = list(range(5,60)))
upper_con = pd.Series([10**func_lin(np.log10(L), con_fit[0] + 2*np.sqrt(con_pcov[0][0]), con_fit[1] - 2*np.sqrt(con_pcov[1][1])) for L in range(5,60)], index = list(range(5,60)))
lower_con = pd.Series([10**func_lin(np.log10(L), con_fit[0] - 2*np.sqrt(con_pcov[0][0]), con_fit[1] + 2*np.sqrt(con_pcov[1][1])) for L in range(5,60)], index = list(range(5,60)))

fig_rates.add_trace(go.Scatter(x = upper_con.index, y = upper_con, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], width = 0), name = 'CI', showlegend = False, legendgroup = 'con_regress'))
fig_rates.add_trace(go.Scatter(x = mean_con.index, y = mean_con, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1]), name = 'Con. PL fit', legendgroup = 'con_regress', fill = 'tonexty'))
fig_rates.add_trace(go.Scatter(x = lower_con.index, y = lower_con, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], width = 0), name = 'CI', showlegend = False,  legendgroup = 'con_regress', fill = 'tonexty'))

fig_rates.update_layout(font=dict(family = 'Arial', size = 14), margin={'t':20,'l':80,'b':40,'r':10}, height = 300, width = 640)

In [ ]:
fig_rates.write_image('plots/fig_rates_regression.pdf')

In [ ]:
exp_var_L9 = ((np.log10(lower_exp[9]) - np.log10(mean_exp[9]))/2)**2
exp_cov_L9 = (exp_pcov[0][1] / (np.sqrt(np.sqrt(exp_pcov[0][0])**2 * np.sqrt(exp_pcov[1][1])**2))) * np.sqrt(exp_pcov[0][0]) * np.sqrt(exp_var_L9)

con_var_L9 = ((np.log10(lower_con[9]) - np.log10(mean_con[9]))/2)**2
con_cov_L9 = (con_pcov[0][1] / (np.sqrt(np.sqrt(con_pcov[0][0])**2 * np.sqrt(con_pcov[1][1])**2))) * np.sqrt(con_pcov[0][0]) * np.sqrt(con_var_L9)

combined_pcov = np.array([[exp_pcov[0][0], exp_cov_L9, 0, 0], [exp_cov_L9, exp_var_L9, 0, 0], [0, 0, con_pcov[0][0], con_cov_L9], [0, 0, con_cov_L9, con_var_L9]])
combined_pcov

#### restrictive prior

In [ ]:
combined_mvg = scipy.stats.multivariate_normal([exp_fit[0], np.log10(mean_exp[9]), con_fit[0], np.log10(mean_con[9])], combined_pcov * 100)

prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

In [ ]:
prob_grid.to_pickle('simulations/grid_4param_v2/prior_restrictive_a.pickle')

#### relaxed prior

In [ ]:
combined_mvg = scipy.stats.multivariate_normal([exp_fit[0], np.log10(mean_exp[9]), con_fit[0], np.log10(mean_con[9])], combined_pcov * 1000)

prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

In [ ]:
prob_grid.to_pickle('simulations/grid_4param_v2/prior_a.pickle')

#### plot prior

In [ ]:
# Fig. S6b
fig = make_subplots(rows = 8, cols = 8, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.005, vertical_spacing = 0.005, row_titles = ['<i>c</i><sub>𝜅</sub>=' + '{:.2e}'.format(x) for x in intercept_list[::-1]], column_titles = ['<i>c</i><sub>𝜀</sub>=' + '{:.2e}'.format(x) for x in intercept_list], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
fig.update_xaxes(range = [0,4.3], tickvals = [0,1,2,3,4], gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
for exp_i in range(8):
    row_count=0; col_count +=1
    for con_i in range(7,-1,-1):
        row_count+=1
        fig.add_trace(go.Heatmap(x = prob_grid.loc[exp_i].loc[con_i].index.get_level_values(0), y = prob_grid.loc[exp_i].loc[con_i].index.get_level_values(1), z=(prob_grid.loc[exp_i].loc[con_i]['prob']), coloraxis='coloraxis1'), row = row_count, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(tickformat = '1.0e', title = 'prior<br>probability', len=0.2)))#, cmin = np.log(prob_grid.replace(0,np.nan).min().min()), cmax= np.log(prob_grid.max().max())))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 900, width = 1200, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.3], tickvals = [0,1,2,3], gridcolor = 'rgba(0,0,0,0.2)')
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/priorplot_4param_permissive.pdf')

In [ ]:
fig = make_subplots(rows = 2, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')

combined_mvg = scipy.stats.multivariate_normal([exp_fit[0], np.log10(mean_exp[9]), con_fit[0], np.log10(mean_con[9])], combined_pcov * 1000)
prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

row_count=1; col_count = 0
for exp_i in range(1,8):
    col_count+=1
    fig.add_trace(go.Heatmap(x = prob_grid.loc[exp_i].loc[exp_i-1].index.get_level_values(0), y = prob_grid.loc[exp_i].loc[exp_i-1].index.get_level_values(1), z=(prob_grid.loc[exp_i].loc[exp_i-1]['prob']), coloraxis='coloraxis1'), row = row_count, col = col_count)

combined_mvg = scipy.stats.multivariate_normal([exp_fit[0], np.log10(mean_exp[9]), con_fit[0], np.log10(mean_con[9])], combined_pcov * 100)
prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

row_count=2; col_count = 0
for exp_i in range(1,8):
    col_count+=1
    fig.add_trace(go.Heatmap(x = prob_grid.loc[exp_i].loc[exp_i-1].index.get_level_values(0), y = prob_grid.loc[exp_i].loc[exp_i-1].index.get_level_values(1), z=(prob_grid.loc[exp_i].loc[exp_i-1]['prob']), coloraxis='coloraxis2'), row = row_count, col = col_count)

In [ ]:
# Fig. S6c
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(tickformat = '1.0e', title = 'prior<br>probability', len=0.7, x = 1, y=.85)))
fig.update_layout(coloraxis2=dict(colorscale='deep_R', colorbar = dict(tickformat = '1.0e', len=0.45, x = 1, y=0.25)))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 310, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)

fig.show()

In [ ]:
fig.write_image('plots/priorplot_3mult_permissive_restrictive.pdf')

## Log curve fit

In [ ]:
def func_log(x, m, c):
    return (10**c)*(np.log(x-7)/np.log(2))**m

In [ ]:
log_exp_fit, log_exp_pcov = curve_fit(func_log, decode_exp_rate['A'].dropna().index, decode_exp_rate['A'].dropna(), sigma = exp_std)
log_con_fit, log_con_pcov = curve_fit(func_log, decode_con_rate['A'].dropna().index, decode_con_rate['A'].dropna(), sigma = con_std)

In [ ]:
log_combined_mvg = scipy.stats.multivariate_normal([log_exp_fit[0], log_exp_fit[1], log_con_fit[0], log_con_fit[1]], combined_pcov*100) # use linear fit pcov
prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [log_combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

In [ ]:
prob_grid.to_pickle('simulations/grid_4param_v2/prior_log_restrictive.pickle')

In [ ]:
log_combined_mvg = scipy.stats.multivariate_normal([log_exp_fit[0], log_exp_fit[1], log_con_fit[0], log_con_fit[1]], combined_pcov*1000) # use linear fit pcov
#log_combined_mvg = scipy.stats.multivariate_normal([log_exp_fit[0], log_exp_fit[1], log_con_fit[0], log_con_fit[1]], combined_pcov*580) # use linear fit pcov

prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in list(range(8)) for con_i in list(range(8))], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [log_combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in intercept_list for con_i in intercept_list]
prob_grid.set_index( ['exp_i', 'con_i', 'exp_p', 'con_p'], inplace=True)
prob_grid /= prob_grid.sum()

In [ ]:
prob_grid.to_pickle('simulations/grid_4param_v2/prior_log_relaxed.pickle')

### Power law only fit

In [ ]:
range_denovo = range(4,9)
range_decode = range(10,16)

In [ ]:
exp_combined = pd.concat([denovo_exp_rate['A'][list(range_denovo)], decode_exp_rate['A'][list(range_decode)]]).dropna()
con_combined = pd.concat([denovo_con_rate['A'][list(range_denovo)], decode_con_rate['A'][list(range_decode)]]).dropna()

exp_combined_poisson = pd.concat([denovo_exp_rate_poisson['A'].unstack()[list(range_denovo)], decode_exp_rate_poisson['A'].unstack()[list(range_decode)]], axis=1).dropna(how = 'all', axis=1)
con_combined_poisson = pd.concat([denovo_con_rate_poisson['A'].unstack()[list(range_denovo)], decode_con_rate_poisson['A'].unstack()[list(range_decode)]], axis=1).dropna(how = 'all', axis=1)

exp_std_combined = exp_combined_poisson.std().dropna()
exp_std_combined.loc[22:] *=100
exp_std_combined.loc[11:12] *=100

con_std_combined = con_combined_poisson.std().dropna()
con_std_combined.loc[22:] *=100
con_std_combined.loc[11:12] *=100

In [ ]:
def func_lin_lam(x, m, loglam):
    return m * (x - loglam) + np.log10(10**-8)
def func_lin_lam_e(x, m, loglam):
    return m * (x - loglam) + np.log10(4*10**-9)
def func_lin_lam_c(x, m, loglam):
    return m * (x - loglam) + np.log10(8*10**-9)

func_lin_lam_e = func_lin_lam
func_lin_lam_c = func_lin_lam

In [ ]:
exp_fit, exp_pcov = curve_fit(func_lin_lam_e, np.log10(exp_combined.index), np.log10(exp_combined), sigma = np.log10(exp_std_combined))
con_fit, con_pcov = curve_fit(func_lin_lam_c, np.log10(con_combined.index), np.log10(con_combined), sigma = np.log10(con_std_combined))

In [ ]:
combined_pcov = np.array([[exp_pcov[0][0], exp_pcov[0][1], 0, 0], [exp_pcov[1][0], exp_pcov[1][1], 0, 0], [0, 0, con_pcov[0][0], con_pcov[0][1]], [0, 0, con_pcov[1][0], con_pcov[1][1]]])
plonly_lambda_list = list(range(5,21))
combined_mvg = scipy.stats.multivariate_normal([exp_fit[0], exp_fit[1], con_fit[0], con_fit[1]], combined_pcov * 100)
prob_grid = pd.DataFrame([(round(exp_p,2), round(con_p,2), exp_i,con_i) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in plonly_lambda_list for con_i in plonly_lambda_list], columns = ['exp_p', 'con_p', 'exp_i', 'con_i'])
prob_grid['prob'] = [combined_mvg.pdf([exp_p, np.log10(exp_i), con_p, np.log10(con_i)]) for exp_p in np.linspace(0,4,41) for con_p in np.linspace(0,4,41) for exp_i in plonly_lambda_list for con_i in plonly_lambda_list]
prob_grid = prob_grid.where(prob_grid['exp_p'] < prob_grid['con_p']).dropna(how = 'all').set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])
prob_grid /= prob_grid.sum()

In [ ]:
prob_grid.to_pickle('simulations/priors/prior_plonly.pickle')